In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from collections import Counter
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import fisher_exact
from scipy import stats
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import kruskal
import scipy
import matplotlib
from statsmodels.stats.multitest import multipletests
import matplotlib.patches as mpatches
import copy
from matplotlib import gridspec
#import matplotlib.colors as colors
from matplotlib.patches import ConnectionPatch
import matplotlib.ticker
from statsmodels.stats.anova import anova_lm
from itertools import product

In [ ]:
### To combine rows
def cat_rows(series):
    return ','.join(list(series.astype("string")))

### Identify high VAF gene mutation in a participant
def gene_high_vaf(row):
    
    if pd.isnull(row["vaf"]) == False:
        if "," in row["vaf"]:
            v=np.array(row["vaf"].split(",")).astype(float)
            all_genes_present=np.array(row["symbol"].split(","))
            p_mut_all=np.array(row["p_mut"].split(","))
            maxind=np.argmax(v)
            if np.unique(all_genes_present).shape[0] == 1:
                gene_vaf=[all_genes_present[maxind],np.max(v),p_mut_all[maxind],all_genes_present[maxind],np.max(v),p_mut_all[maxind]]
            else:
                gene_vaf=["NA","NA","NA",all_genes_present[maxind],np.max(v),p_mut_all[maxind]]
        else:
            gene_vaf=[row["symbol"],float(row["vaf"]),row["p_mut"],row["symbol"],float(row["vaf"]),row["p_mut"]]
    else:
        gene_vaf=["Control","NA","NA","Control","NA","NA"]
    return gene_vaf

### categorize VAF to high/low based on a single cut-off
def vaf_class(vaf):
    if vaf!="NA" and vaf != np.nan:
        vaf_group=[]
        for t in [0.05,0.10,0.15,0.20,0.25]:
            if float(vaf)>=t:
                vaf_group.append(1)
            else:
                vaf_group.append(0)
        return vaf_group
    else:
            return [vaf,vaf,vaf,vaf,vaf]


In [ ]:
############# Load UKB baseline data ##############
ukb500k=pd.read_csv("/home/sruthi/ukbb_prediction_model/data/ukb500_pheno_prediction.tsv",delimiter="\t",header='infer',low_memory=False,parse_dates=['daac','AML', 'MDS', 'MPN', 'CMML', 'dod'],dtype={"eid": 'string'})
#Remove samples for which somatic variant calling failed
ids_somatic=np.loadtxt('/home/sruthi/ukbb_prediction_model/rap/germline_variants/farm/ddx41/code/ids.txt',dtype="str")

ukb500k=ukb500k[ukb500k["eid"].isin(ids_somatic)]
print(ukb500k.shape)

In [ ]:
######## Add information on genetic sex and exclude people with non-matching genetic and self-reported sex ###################
genetic_sex=pd.read_csv("/home/sruthi/telomere/inputs/genetic_sex.tsv",sep="\t",dtype="str")
genetic_sex.columns=["eid","genetic_sex"]
ukb500k=ukb500k.merge(genetic_sex,how="left",on="eid")

print(ukb500k.genetic_sex.isna().value_counts())
ukb500k=ukb500k[(ukb500k["sex"]==ukb500k["genetic_sex"]) | (ukb500k.genetic_sex.isna())]
ukb500k.shape

In [ ]:
############ Chemotherapy/radiotherapy/malignancy phenotypes
chemo_radio=pd.read_csv("../inputs/ukb_cheracan.tsv",delimiter="\t",dtype={"eid":"string"},parse_dates=["chemo_date","radio_date","All_malignant_neoplasms"],usecols=["eid","chemo_date","radio_date","All_malignant_neoplasms"])
ukb500k=ukb500k.merge(chemo_radio,how="left",on="eid")

############ Add myeloid malignancy prevalence 
mm_prevalence=pd.read_csv("/home/sruthi/Downloads/prevalent_diagnoses.tsv",delimiter="\t",dtype={"eid":'string'})
ukb500k=ukb500k.drop(columns=["AML","MPN","MDS","CMML"])
ukb500k=ukb500k.merge(mm_prevalence,on="eid",how="left",suffixes=["","_b"])

In [ ]:
######## Somatic variants in U2AF1 from pileup ############
u2af1=pd.read_csv("/home/sruthi/ukbb_prediction_model/rap/hotspot_pileup/u2af1/analysis_results/u2af1_variants.tsv",delimiter="\t",usecols=["eid","spdi_x","vaf_total","HGVSp"],dtype={"eid":"str"})
u2af1["symbol"]="U2AF1"
print(u2af1.shape)

u2af1=u2af1.rename(columns={"spdi_x":"c_mut","HGVSp":"p_mut","vaf_total":"vaf"})
u2af1=u2af1[u2af1["vaf"]>=0.05]
u2af1.shape

In [ ]:
################ Somatic variant calls from Mutect2 #############
som=pd.read_csv("/home/sruthi/ukbb_prediction_model/rap/germline_variants/farm/ddx41/extracted_variants/results_var_ch_all_allgenes_UKB",delimiter="\t",header='infer',low_memory=False,dtype={"eid": 'string'})
som["p_mut"]=som["p_mut"].fillna(value="NA")

################ Somatic variant calls of CHEK2
chek2=pd.read_csv("/home/sruthi/telomere/code/chek2_vep/chek2_filtered_variants_simple.tsv",delimiter="\t",dtype={"eid":"str"},usecols=["eid","HGVSp","HGVSc","vaf"])
chek2["symbol"]="CHEK2"
chek2=chek2.rename(columns={"HGVSc":"c_mut","HGVSp":"p_mut"})


genes_to_include=["ASXL1","ATM","BCOR","BCORL1","BRAF","BRCC3","CALR","CBL","DNMT3A","EZH2","FLT3","GNAS","GNB1","IDH1","IDH2","JAK2","KDM6A","KIT","KRAS","MPL","NPM1","NRAS","PHF6","PIGA","PPM1D","PRPF40B","RAD21","RUNX1","SF1","SF3A1","SF3B1","SMC1A","SMC3","SRSF2","STAG2","STAT3","TET2","TP53","U2AF1","U2AF2","ZRSR2","CHEK2"]


print(som.shape)
som=som[som["symbol"]!="U2AF1"]
print(som.shape)
som=pd.concat([som,u2af1,chek2],axis=0)
print(som.shape)
som=som[som["symbol"].isin(genes_to_include)]
print(som.shape)
som_comb=som.groupby('eid')[['symbol',"vaf","c_mut","p_mut"]].agg(cat_rows).reset_index()


In [ ]:
########### Telomere length information #############
telomere_length=pd.read_csv("../inputs/ts_ratios.tsv",delimiter="\t",header='infer',low_memory=False,dtype={"f.eid": 'string'},usecols=["f.eid","f.22190.0.0","f.22191.0.0","f.22192.0.0","f.22191.1.0","f.22191.2.0"])

In [ ]:
########### Combine baseline, telomere length and somatic variants data ##########
ukb500k_ts_som=ukb500k.merge(telomere_length,left_on="eid",right_on="f.eid",how="left")
ukb500k_ts_som=ukb500k_ts_som.merge(som_comb,on="eid",how="left")
ukb500k_ts_som=ukb500k_ts_som.rename(columns={"f.22190.0.0":"Unadjusted T/S ratio","f.22191.0.0":"Adjusted T/S ratio","f.22192.0.0":"Z-adjusted T/S log","f.22191.1.0":"Adjusted T/S ratio 1","f.22191.2.0":"Adjusted T/S ratio 2"})

In [ ]:
############ UKB age bins
ukb500k_ts_som["age_bins"]=pd.cut(ukb500k_ts_som["age"],bins=np.arange(36,77,4))

In [ ]:
########### Remove participants with blood collection date different from DAAC and participants who withdrew consent
samples_not_matching_daac=np.loadtxt("/home/sruthi/reference/ukbiobank/450k_exomes/blood_sample_collection_date/samples_not_matching.tsv",usecols=[0],dtype="str")

samples_no_consent=np.loadtxt("/home/sruthi/reference/ukbiobank/withdraw_consent_eids/w56844_2023-04-25.csv", dtype="str")

ukb500k_ts_som=ukb500k_ts_som[~((ukb500k_ts_som["eid"].isin(samples_not_matching_daac)) | 
ukb500k_ts_som["eid"].isin(samples_no_consent))]
ukb500k_ts_som.shape

In [ ]:
#som.loc[som["eid"].isin(ukb500k_ts_som["eid"]),['symbol',"vaf","c_mut","p_mut"]].to_csv("somatic_calls.tsv",sep="\t",index=False)

In [ ]:
som.loc[(som["symbol"].isin(["ASXL1","CALR","CBL","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","DNMT3A","DNMT3A","U2AF1"])) & (som["eid"].isin(ukb500k_ts_som["eid"])),"vaf"].shape,som.loc[(som["symbol"].isin(["ASXL1","CALR","CBL","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","DNMT3A","DNMT3A","U2AF1"])) & (som["eid"].isin(ukb500k_ts_som["eid"])) & (som["vaf"].astype(float)>0.02),"vaf"].shape

In [ ]:
som.loc[((som["eid"].isin(ukb500k_ts_som["eid"])) & (som["vaf"].astype(float)>0.02)),"vaf"].shape, som.loc[(som["eid"].isin(ukb500k_ts_som["eid"])),"vaf"].shape

In [ ]:
############ Age related prevalence of CH genes
prev=ukb500k_ts_som["age"].value_counts().rename("t")
ch_prev=[]
for i,gene in enumerate(["DNMT3A", "TET2", "ASXL1", "PPM1D", "TP53", "JAK2", "SRSF2", "SF3B1", "GNB1","GNAS","U2AF1"]):
    if gene!="Control":
        ukb500k_ts_som["prev"]="UKB"
        ukb500k_ts_som.loc[ukb500k_ts_som["symbol"].str.contains(gene,na=False),"prev"]=gene
        gene_prev=pd.concat([prev,ukb500k_ts_som.loc[ukb500k_ts_som["prev"]==gene,"age"].value_counts().rename("n")],axis=1)
        gene_prev["pct"]=gene_prev["n"]/gene_prev["t"]*100
        gene_prev["single_gene"]=gene
        print(gene_prev)
        ch_prev.append(gene_prev)
ch_prev=pd.concat(ch_prev)
ch_prev=ch_prev.reset_index().rename(columns={"index":"age"})


In [ ]:
ukb_age=ukb500k_ts_som[["eid","age","sex"]].copy(deep=True)
#ch_prev.to_csv("ch_prevalence.tsv",sep="\t",index=False,na_rep="NA")

In [ ]:
############ Mark somatic variants with highest VAF in each participant and group participants based on VAF ############
ukb500k_ts_som[["single_gene","single_gene_vaf","single_gene_pmut","multi_gene","multi_gene_vaf","multi_gene_pmut"]]=ukb500k_ts_som.apply(lambda row: gene_high_vaf(row),axis=1,result_type='expand')
#ukb500k_ts_som[["single_gene","single_gene_vaf","single_gene_pmut"]]=ukb500k_ts_som.apply(lambda row: gene_high_vaf(row),axis=1,result_type='expand')

ukb500k_ts_som[["multi_gene_vaf_0.05","multi_gene_vaf_0.10","multi_gene_vaf_0.15","multi_gene_vaf_0.20","multi_gene_vaf_0.25"]]=ukb500k_ts_som.apply(lambda row:vaf_class(row["multi_gene_vaf"]),axis=1,result_type='expand')
ukb500k_ts_som[["single_gene_vaf_0.05","single_gene_vaf_0.10","single_gene_vaf_0.15","single_gene_vaf_0.20","single_gene_vaf_0.25"]]=ukb500k_ts_som.apply(lambda row:vaf_class(row["single_gene_vaf"]),axis=1,result_type='expand')

In [ ]:
############ Calculate polygenic risk scores - UKB-GWAS,topmed-GWAS############
directories =["/home/sruthi/telomere/polygenic_risk_score/keep_ambig/outfiles/all_chr_prs_files","/home/sruthi/telomere/polygenic_risk_score/topmed/outfiles_all_trans/all_chr_prs_files"]
column_names=["total_PRS","topmed_all_trans_total_PRS"]
prs_scores=[]
for d,directory in enumerate(directories):
    i=1
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".all_score"):
            print (i,filename)
            f=pd.read_csv(directory+"/"+filename,usecols=["IID","Pt_1"],dtype={"IID":"string"},header="infer",sep=" ")
            if i==1:
                prs=f
                prs=prs.rename(columns={"Pt_1":"total_PRS"})
            else:
                prs=prs.merge(f,on="IID",how="inner")
                prs["total_PRS"]=prs["total_PRS"]+prs["Pt_1"]
                prs=prs[["IID","total_PRS"]]
            i+=1
    prs.columns=["eid",column_names[d]]
    prs_scores.append(prs.set_index("eid"))
prs=pd.concat(prs_scores,axis=1,join='outer').reset_index()

ukb500k_ts_som=ukb500k_ts_som.merge(prs,on="eid",how="left")
ukb500k_ts_som.shape

In [ ]:
########### Split DNMT3A into DNMT3A_R882 AND DNMT3A_other
ukb500k_ts_som.loc[((ukb500k_ts_som["single_gene"]=="DNMT3A") & (ukb500k_ts_som["single_gene_pmut"].str.contains("R882"))),"single_gene"]="DNMT3A_R882"
ukb500k_ts_som.loc[(ukb500k_ts_som["single_gene"]=="DNMT3A"),"single_gene"]="DNMT3A_other"
ukb500k_ts_som["single_gene"].value_counts()

ukb500k_ts_som.loc[((ukb500k_ts_som["multi_gene"]=="DNMT3A") & (ukb500k_ts_som["multi_gene_pmut"].str.contains("R882"))),"multi_gene"]="DNMT3A_R882"
ukb500k_ts_som.loc[(ukb500k_ts_som["multi_gene"]=="DNMT3A"),"multi_gene"]="DNMT3A_other"
ukb500k_ts_som["multi_gene"].value_counts()


In [ ]:
ukb500k_ts_som["single_gene"].value_counts()


In [ ]:
########## Genes to include in the analysis ###########
imp_genes=["ASXL1","CALR","CBL","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","Control","DNMT3A_other","DNMT3A_R882","U2AF1"]
all_genes=["Control","DNMT3A_other","TET2","DNMT3A_R882","ASXL1","PPM1D","TP53","JAK2","SF3B1","SRSF2","GNB1","GNAS","CBL","ATM","STAT3","CALR","BRCC3","KDM6A","KRAS","BCOR", "STAG2","BCORL1","U2AF1", "IDH2","CHEK2"]

In [ ]:
######### Chemotherapy/radiotherapy/malignant neoplasm prevalence in PPM1D mutation carriers
print(ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D")].shape,ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D") & (ukb500k_ts_som["chemo_date"]<ukb500k_ts_som["daac"])].shape,ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D") & ((ukb500k_ts_som["chemo_date"].isna()) | (ukb500k_ts_som["chemo_date"]>ukb500k_ts_som["daac"]))].shape)

print(ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D")].shape,ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D") & (ukb500k_ts_som["radio_date"]<ukb500k_ts_som["daac"])].shape,ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D") & ((ukb500k_ts_som["radio_date"].isna()) | (ukb500k_ts_som["radio_date"]>ukb500k_ts_som["daac"]))].shape)

print(ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D")].shape,ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D") & (ukb500k_ts_som["All_malignant_neoplasms"]<ukb500k_ts_som["daac"])].shape,ukb500k_ts_som[(ukb500k_ts_som["single_gene"]=="PPM1D") & ((ukb500k_ts_som["All_malignant_neoplasms"].isna()) | (ukb500k_ts_som["All_malignant_neoplasms"]>ukb500k_ts_som["daac"]))].shape)

print(ukb500k_ts_som[ukb500k_ts_som["single_gene"].str.contains("PPM1D",na=False)].shape,ukb500k_ts_som[ukb500k_ts_som["single_gene"].str.contains("PPM1D",na=False) & (((ukb500k_ts_som["chemo_date"].notna()) & (ukb500k_ts_som["chemo_date"]<ukb500k_ts_som["daac"])) | ((ukb500k_ts_som["radio_date"].notna()) & (ukb500k_ts_som["radio_date"]<ukb500k_ts_som["daac"])) | ((ukb500k_ts_som["All_malignant_neoplasms"].notna()) & (ukb500k_ts_som["All_malignant_neoplasms"]<ukb500k_ts_som["daac"])))].shape)

ukb500k_ts_som[(ukb500k_ts_som["single_gene"].str.contains("PPM1D",na=False)) & (((ukb500k_ts_som["chemo_date"].isna()) | (ukb500k_ts_som["chemo_date"]>ukb500k_ts_som["daac"])) & ((ukb500k_ts_som["radio_date"].isna()) | (ukb500k_ts_som["radio_date"]>ukb500k_ts_som["daac"])) & ((ukb500k_ts_som["All_malignant_neoplasms"].isna()) | (ukb500k_ts_som["All_malignant_neoplasms"]>ukb500k_ts_som["daac"])))].shape


In [ ]:
##########Identify people with SRSF2,TET2 and DNMT3A,TET2 mutations for the double mutant analysis
ukb500k_ts_som["symbol_unique"]=ukb500k_ts_som["symbol"].map(lambda row:",".join([gene for gene in np.unique(np.array(row.split(",")))]),na_action='ignore')

print(ukb500k_ts_som["symbol_unique"].value_counts()["SRSF2,TET2"],ukb500k_ts_som["symbol_unique"].value_counts()["DNMT3A,TET2"])

ukb500k_ts_som["multi_gene_TET2"]=ukb500k_ts_som.apply(lambda row:row["symbol_unique"]+"-"+row["multi_gene"] if (row["symbol_unique"]=="SRSF2,TET2" or row["symbol_unique"]=="DNMT3A,TET2") else row["multi_gene"],axis=1)
ukb500k_ts_som["multi_gene_TET2_vaf"]=ukb500k_ts_som.apply(lambda row:row["multi_gene_vaf"] if (row["symbol_unique"]=="SRSF2,TET2" or row["symbol_unique"]=="DNMT3A,TET2")else row["multi_gene_vaf"],axis=1)
print(ukb500k_ts_som.shape)

ukb500k_ts_som.loc[(ukb500k_ts_som["multi_gene_TET2"]=="DNMT3A,TET2-DNMT3A_other"),"multi_gene_TET2"]="DNMT3A,TET2-DNMT3A"
ukb500k_ts_som.loc[(ukb500k_ts_som["multi_gene_TET2"]=="DNMT3A,TET2-DNMT3A_R882"),"multi_gene_TET2"]="DNMT3A,TET2-DNMT3A"


In [ ]:
########### Data with all genes, remove duplicate columns, remove mutations in less frequent genes ###############
ukb500k_ts_som_allgenes=ukb500k_ts_som.copy(deep=True)
ukb500k_ts_som_allgenes=ukb500k_ts_som_allgenes.loc[:,(ukb500k_ts_som_allgenes.columns!="IID")]
ukb500k_ts_som_allgenes=ukb500k_ts_som_allgenes.loc[:,(ukb500k_ts_som_allgenes.columns!="f.eid")]
ukb500k_ts_som_allgenes=ukb500k_ts_som_allgenes[(ukb500k_ts_som_allgenes["single_gene"].isin(all_genes)) & ((ukb500k_ts_som_allgenes["total_PRS"].notna()) | (ukb500k_ts_som_allgenes["Z-adjusted T/S log"].notna()))]
ukb500k_ts_som_allgenes.shape

In [ ]:
########### Data with main genes (highest VAF), remove duplicate columns, remove mutations in less frequent genes  ###############
ukb500k_ts_som_multigene=ukb500k_ts_som.copy(deep=True)
ukb500k_ts_som_multigene=ukb500k_ts_som_multigene.loc[:,(ukb500k_ts_som_multigene.columns!="IID")]
ukb500k_ts_som_multigene=ukb500k_ts_som_multigene.loc[:,(ukb500k_ts_som_multigene.columns!="f.eid")]
ukb500k_ts_som_multigene=ukb500k_ts_som_multigene[(ukb500k_ts_som_multigene["multi_gene"].isin(imp_genes)) & ((ukb500k_ts_som_multigene["total_PRS"].notna()) | (ukb500k_ts_som_multigene["Z-adjusted T/S log"].notna()))]
ukb500k_ts_som_multigene.shape

In [ ]:
############ Data with main genes (single mutation only), remove duplicate columns, remove mutations in less frequent genes #############
ukb500k_ts_som=ukb500k_ts_som.loc[:,(ukb500k_ts_som.columns!="IID")]
ukb500k_ts_som=ukb500k_ts_som.loc[:,(ukb500k_ts_som.columns!="f.eid")]
ukb500k_ts_som=ukb500k_ts_som[(ukb500k_ts_som["single_gene"].isin(imp_genes)) & ((ukb500k_ts_som["total_PRS"].notna()) | (ukb500k_ts_som["Z-adjusted T/S log"].notna()))]
ukb500k_ts_som.shape

In [ ]:
colors=sns.color_palette('husl',len(imp_genes))
print(len(colors))
gene_color={'Control':colors[0]}
for i,gene in enumerate(sorted([x for x in imp_genes if x!="Control"])):
    gene_color[gene]=colors[i+1]

colors

colors=sns.color_palette('husl',len(all_genes))
print(len(colors))
gene_color_allgenes={'Control':colors[0]}
for i,gene in enumerate(sorted([x for x in all_genes if x!="Control"])):
    gene_color_allgenes[gene]=colors[i+1]

In [ ]:
########### Z-normalize PRS #################
for y in ["total_PRS","topmed_all_trans_total_PRS"]:
    ukb500k_ts_som[y+"_old"]=ukb500k_ts_som[y]
    ukb500k_ts_som[y]=(ukb500k_ts_som[y]-ukb500k_ts_som.loc[(ukb500k_ts_som["single_gene"]=="Control"),y].mean())/np.std(ukb500k_ts_som.loc[(ukb500k_ts_som["single_gene"]=="Control"),y])
    
    ukb500k_ts_som_allgenes[y+"_old"]=ukb500k_ts_som_allgenes[y]
    ukb500k_ts_som_allgenes[y]=(ukb500k_ts_som_allgenes[y]-ukb500k_ts_som_allgenes.loc[(ukb500k_ts_som_allgenes["single_gene"]=="Control"),y].mean())/np.std(ukb500k_ts_som_allgenes.loc[(ukb500k_ts_som_allgenes["single_gene"]=="Control"),y])
    
    ukb500k_ts_som_multigene[y+"_old"]=ukb500k_ts_som_multigene[y]
    ukb500k_ts_som_multigene[y]=(ukb500k_ts_som_multigene[y]-ukb500k_ts_som_multigene.loc[(ukb500k_ts_som_multigene["single_gene"]=="Control"),y].mean())/np.std(ukb500k_ts_som_multigene.loc[(ukb500k_ts_som_multigene["single_gene"]=="Control"),y])

In [ ]:
############# Winsorize WBC percentages ##########
for wbc_perc in ["WBC","LY_perc","MO_perc","NE_perc","EO_perc","BA_perc"]:
    ukb500k_ts_som[wbc_perc+"_wins"]=scipy.stats.mstats.winsorize(ukb500k_ts_som[wbc_perc],limits=[0.005,0.005],nan_policy="omit")
    ukb500k_ts_som_allgenes[wbc_perc+"_wins"]=scipy.stats.mstats.winsorize(ukb500k_ts_som_allgenes[wbc_perc],limits=[0.005,0.005],nan_policy="omit")
    ukb500k_ts_som_multigene[wbc_perc+"_wins"]=scipy.stats.mstats.winsorize(ukb500k_ts_som_multigene[wbc_perc],limits=[0.005,0.005],nan_policy="omit")

In [ ]:
ukb500k_ts_som.shape,ukb500k_ts_som_allgenes.shape

In [ ]:
ukb500k_ts_som["total_PRS"].min(),ukb500k_ts_som["total_PRS"].max(),ukb500k_ts_som["Z-adjusted T/S log"].min(),ukb500k_ts_som["Z-adjusted T/S log"].max()

In [ ]:
########    Add mCA data   ########################################
mca=pd.read_csv("/home/sruthi/reference/ukbiobank/450k_exomes/mCA/CNV_data_19808.txt", usecols=["ID","CHR","START_MB","END_MB","SIZE_MB","COPY_CHANGE",   "CELL_FRAC"],dtype="string",header='infer',delimiter="\t")
display(mca)

bridge_mca=pd.read_csv("/home/sruthi/reference/ukbiobank/450k_exomes/mCA/ukb56844bridge19808.txt",dtype="str",delimiter=" ",names=["eid","eid_return"])
display(bridge_mca)

mca=mca.merge(bridge_mca,left_on="ID",right_on="eid_return",how="inner")
print(mca.eid_return.isna().value_counts())
display(mca)

mca=mca.loc[:,(mca.columns!="eid_return")]
display(mca)

mca_samples_included=pd.read_csv("/home/sruthi/reference/ukbiobank/450k_exomes/mCA/ukb4777.samples_analyzed.txt",dtype="str",delimiter="\t",names=["eid_return"])
mca_samples_included=mca_samples_included.merge(bridge_mca,on="eid_return",how="left")
print(mca_samples_included.eid_return.isna().value_counts())

mca["CHR_COPY_CHANGE"]=mca.apply(lambda row:row["CHR"]+":"+row["COPY_CHANGE"],axis=1)

mca_comb=mca.groupby('eid').agg(cat_rows).reset_index()
ukb_mca_age=ukb_age.merge(mca_comb,on="eid",how="left")
ukb_mca_age=ukb_mca_age[ukb_mca_age["eid"].isin(mca_samples_included["eid"])]

#mca_comb["CHR_COPY_CHANGE"]=mca_comb.apply(lambda row:row["CHR"]+":"+row["COPY_CHANGE"] if "," not in row["COPY_CHANGE"] else "NA",axis=1)
mca_comb["CHR_COPY_CHANGE"]=mca_comb["CHR_COPY_CHANGE"].map(lambda row:row if "," not in row else "NA")

display(mca["CHR"].value_counts())

In [ ]:
# Combine with somatic variant call data
ukb500k_ts_som=ukb500k_ts_som.merge(mca_comb,on="eid",how="left")

ukb500k_ts_som_mca=ukb500k_ts_som[ukb500k_ts_som["eid"].isin(mca_samples_included["eid"])].copy(deep=True)
print(ukb500k_ts_som_mca.shape)
ukb500k_ts_som_mca.loc[ukb500k_ts_som_mca["CHR_COPY_CHANGE"].isna(),"CHR_COPY_CHANGE"]="Control"
ukb500k_ts_som_mca=ukb500k_ts_som_mca[(~(ukb500k_ts_som_mca["CHR_COPY_CHANGE"].str.contains("unknown",na=False)) & (ukb500k_ts_som_mca["CHR_COPY_CHANGE"]!="NA")  & (ukb500k_ts_som_mca["single_gene"]=="Control"))]

print(ukb500k_ts_som.shape,ukb500k_ts_som_mca.shape)

ukb500k_ts_som_mca=ukb500k_ts_som_mca[((ukb500k_ts_som_mca["CELL_FRAC"]!="unknown") & ~(ukb500k_ts_som_mca["CELL_FRAC"].str.contains(","))) | (ukb500k_ts_som_mca["CELL_FRAC"].isna())]
print(ukb500k_ts_som_mca.shape)
ukb500k_ts_som_mca["CELL_FRAC"]=ukb500k_ts_som_mca["CELL_FRAC"].astype(float)

chromosomes=[str(i) for i in range(1,23)]+["X","Y"]
possible_MCAs=[i+":"+"loss" for i in chromosomes]+[i+":"+"neutral" for i in chromosomes]+[i+":"+"gain" for i in chromosomes]

In [ ]:
########## Calculate prevalence of mCA #################           
def check_autosome(row):
    autosome=[i+":"+"loss" for i in chromosomes[0:22]]+[i+":"+"neutral" for i in chromosomes[0:22]]+[i+":"+"gain" for i in chromosomes[0:22]]#+[i+":"+"unknown" for i in chromosomes[0:22]]
    for a in autosome:
        if a in str(row).split(","):
            return "autosomal"
    return row

mca_prev_all=[]
                                                                                     
for m in ["X:loss","Y:loss","autosomal"]:
    ukb_mca_age["single_gene"]="UKB"                                                                        
    if m=="autosomal":
        ukb_mca_age["single_gene"]=ukb_mca_age["CHR_COPY_CHANGE"].map(lambda row:check_autosome(row))
        mca_prev=pd.concat([ukb_mca_age["age"].value_counts().rename("t"),ukb_mca_age.loc[ukb_mca_age["single_gene"]==m,"age"].value_counts().rename("n")],axis=1)
        mca_prev["pct"]=mca_prev["n"]/mca_prev["t"]*100
        mca_prev["single_gene"]="autosomal"
            
    elif m=="Y:loss":
        temp=ukb_mca_age.loc[(ukb_mca_age["sex"]=="Male")].copy(deep=True)
        temp["single_gene"]=temp["CHR_COPY_CHANGE"].map(lambda row:m if m in str(row).split(",") else row)
        mca_prev=pd.concat([ukb_mca_age.loc[(ukb_mca_age["sex"]=="Male"),"age"].value_counts().rename("t"),temp.loc[temp["single_gene"]==m,"age"].value_counts().rename("n")],axis=1)
        mca_prev["pct"]=mca_prev["n"]/mca_prev["t"]*100
        mca_prev["single_gene"]="LOY"
        
    else:
        temp=ukb_mca_age.loc[(ukb_mca_age["sex"]=="Female")].copy(deep=True)
        temp["single_gene"]=temp["CHR_COPY_CHANGE"].map(lambda row:m if m in str(row).split(",") else row)
        mca_prev=pd.concat([ukb_mca_age.loc[(ukb_mca_age["sex"]=="Female"),"age"].value_counts().rename("t"),temp.loc[temp["single_gene"]==m,"age"].value_counts().rename("n")],axis=1)
        mca_prev["pct"]=mca_prev["n"]/mca_prev["t"]*100
        mca_prev["single_gene"]="LOX"

    mca_prev_all.append(mca_prev)
mca_prev_all=pd.concat(mca_prev_all) 
mca_prev_all=mca_prev_all.reset_index().rename(columns={"index":"age"})                                                                                                                            

In [ ]:
#mca_prev_all.to_csv("mca_prevalence.tsv",sep="\t",index=False,na_rep="NA")

In [ ]:
################ PRS for mCA types ###############
temp=ukb500k_ts_som_mca[["total_PRS","sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","CHR_COPY_CHANGE"]].copy(deep=True)
print(temp.shape)
temp=temp[~(temp["CHR_COPY_CHANGE"].isin(["Y:neutral","X:neutral","Y:gain","X:gain"]))]
temp=temp.dropna()
temp["MCA_type"]=temp["CHR_COPY_CHANGE"].map(lambda row:"autosomal MCA" if row not in ["Y:loss","X:loss","Control"] else row)
y="total_PRS"

counts=temp["MCA_type"].value_counts()
print(counts.sum())
gene_order=["autosomal MCA","Y:loss","X:loss"]
plt.figure(figsize=(6,4))
b=sns.violinplot(data=temp,x="MCA_type",y=y,order=["Control"]+gene_order,linewidth=2,palette={"Control":gene_color["Control"],"autosomal MCA":"#999999","Y:loss":"#E69F00","X:loss":"#56B4E9"})
ax=plt.gca()
xticks=ax.get_xticklabels()
xticks_new=[]
for i in xticks:
    print(i.get_text(),counts[i.get_text()])
    xticks_new.append(i.get_text()+"\n("+str(counts[i.get_text()])+")")
#plt.ylim([-5,5])
plt.xticks(np.arange(0,len(xticks),1),xticks_new)
plt.axhline(y=temp.loc[temp["MCA_type"]=="Control",y].median(), color='k', linestyle='--')
plt.grid(axis='x')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.xaxis.label.set_size(18)
ax.yaxis.label.set_size(16)
plt.ylabel("PRS")
plt.xlabel("")
plt.show()
            

In [ ]:
################### Association analysis for mCA-PRS #####################
plot_values=[]
for mca_type in ["autosomal MCA","Y:loss","X:loss"]:
    if mca_type=="autosomal MCA":
        data=temp[temp["MCA_type"].isin(["Control",mca_type])].copy(deep=True)
        print(data.shape)
        formula = 'MCA_type ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"total_PRS"])
    elif mca_type=="Y:loss":
        data=temp[(temp["MCA_type"].isin(["Control",mca_type])) & (temp["sex"]=="Male")].copy(deep=True)
        formula = 'MCA_type ~' + ' + '.join(['age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"total_PRS"])
    else:
        data=temp[(temp["MCA_type"].isin(["Control",mca_type])) & (temp["sex"]=="Female")].copy(deep=True)
        formula = 'MCA_type ~' + ' + '.join(['age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"total_PRS"])
    data["MCA_type"]=data["MCA_type"].map({mca_type:1,"Control":0})
    data["MCA_type"]=data["MCA_type"].astype(int)
    
    log_model=smf.logit(formula=formula,data=data)
    result=log_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = np.exp(result.params)
    conf = np.exp(result.conf_int())
    print(result.pvalues)
    plot_values.append([params["total_PRS"],conf.loc["total_PRS",0],conf.loc["total_PRS",1],result.pvalues["total_PRS"]])

plot_values=np.array(plot_values)
data_mca=temp.copy(deep=True)
data_mca=data_mca.rename(columns={"Z-adjusted T/S log":"tl"})

In [ ]:
plt.errorbar(np.arange(3),plot_values[:,0],yerr=[plot_values[:,0]-plot_values[:,1],plot_values[:,2]-plot_values[:,0]],fmt="o",markersize=7)
ticklabel=["autosomal MCA","Y:loss","X:loss"]
ticklabel=[ticklabel[i]+"\n*" if plot_values[i,3]<0.05 else ticklabel[i] for i in range(3)]
plt.xticks(np.arange(3),ticklabel,fontsize=13)
plt.ylabel("OR",fontsize=13)
plt.axhline(y=1,linestyle="--",lw=1)


In [ ]:
########### Save data ################                                                                                                                                                                                                                                                                                                                                                       
supptable_1_mca=pd.DataFrame(plot_values)
supptable_1_mca["Gene/mCA"]=["autosomal mCA","LOY","LOX"]
supptable_1_mca.columns=["OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value","Gene/mCA"]
supptable_1_mca=supptable_1_mca[["Gene/mCA","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value"]]
supptable_1_mca["FDR"]=multipletests(supptable_1_mca["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
#supptable_1_mca[["Gene/mCA","Coefficient","Lower CI (2.5%)","Upper CI (97.5%)","P-value"]].to_csv("MCA_coef_supptable.csv",sep="\t",index=False)

In [ ]:
############# Association with all CH and PRS ###############
y="total_PRS"
temp=ukb500k_ts_som.copy(deep=True)
temp=temp.loc[(temp[y].notna())]

formula = 'single_gene ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','total_PRS'])

or_table_pr=[]

data=temp[["total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10", "single_gene"]].copy(deep=True)
data=data.dropna()
print(data.shape)
data["single_gene"]=data["single_gene"].map(lambda row:0 if row=="Control" else 1)
data["single_gene"]=data["single_gene"].astype(int)
print(data["single_gene"].value_counts())
log_model_allch=smf.logit(formula=formula,data=data)
result_allch=log_model_allch.fit(max_iterations= 100000)
display(result_allch.summary2())
params_allch = np.exp(result_allch.params)
conf_allch = np.exp(result_allch.conf_int())
print(result_allch.pvalues)
print(["all CH", params_allch["total_PRS"], conf_allch.loc["total_PRS",0], conf_allch.loc["total_PRS",1], result_allch.pvalues["total_PRS"]])
or_table_pr.append(["all CH", params_allch["total_PRS"], conf_allch.loc["total_PRS",0], conf_allch.loc["total_PRS",1], result_allch.pvalues["total_PRS"]])


In [ ]:
############# Association with CH and PRS ###############

y="total_PRS"
temp=ukb500k_ts_som.copy(deep=True)
temp=temp.loc[(temp[y].notna())]

formula = 'single_gene ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','total_PRS'])
gene_list=list(temp["single_gene"].unique())
gene_list.remove("Control")

for gene in gene_list:
    data=temp[["total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","single_gene"]].copy(deep=True)
    data=data.dropna()
    print(data.shape)
    data=data[(data["single_gene"].isin([gene,"Control"]))]
    data["single_gene"]=data["single_gene"].map({gene:1,"Control":0})
    data["single_gene"]=data["single_gene"].astype(int)
    print(data["single_gene"].value_counts())
    log_model=smf.logit(formula=formula,data=data)
    result=log_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = np.exp(result.params)
    conf = np.exp(result.conf_int())
    print(result.pvalues)
    print([gene,params["total_PRS"],conf.loc["total_PRS",0],conf.loc["total_PRS",1],result.pvalues["total_PRS"]])
    or_table_pr.append([gene,params["total_PRS"],conf.loc["total_PRS",0],conf.loc["total_PRS",1],result.pvalues["total_PRS"]])
or_table_pr=pd.DataFrame(or_table_pr)

In [ ]:
or_table_pr

In [ ]:
## Plot Figure 1b

y="total_PRS"
temp=ukb500k_ts_som[["total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","single_gene"]].copy(deep=True)
temp=temp.dropna()
gene_count=Counter(temp["single_gene"])

left, width = 0.01, 15*0.05
bottom_h, height_h=0.1,0.23
bottom, height = 0.35, .60
left_h = left+width+0.03
rect_0 = [left, bottom, width, height]
rect_1 = [left_h, bottom, 4*0.05, height]
rect_2 = [left, bottom_h, width, height_h]
rect_3 = [left_h, bottom_h, 4*0.05, height_h]

plt.figure(figsize=(22,12))
ax1=plt.axes(rect_0)
ax2=plt.axes(rect_1)
ax3=plt.axes(rect_2,sharex=ax1)
ax = [ax1, ax2, ax3, plt.axes(rect_3,sharex=ax2)]

gene_color_new={}
for key in gene_color:
    gene_color_new[key]=sns.set_hls_values(gene_color[key],s=0.5)
black=sns.set_hls_values('#000000',s=0.5)

gene_list=[]
gene_groups=[]
quartiles=[]

gene_list.append("Control")
gene_groups.append(np.array(temp.loc[((temp["single_gene"]=="Control") & (temp[y].notna())),y]))
quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))

gene_list.append("all CH")
gene_groups.append(np.array(temp.loc[((temp["single_gene"].isin(["ASXL1","CALR","CBL","DNMT3A_R882","DNMT3A_other","GNAS", "GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1"])) & (temp[y].notna())),y]))
quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))

for key in ["ASXL1","CALR","CBL","DNMT3A_R882","DNMT3A_other","GNAS", "GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1"]: 
        if gene_count[key]>=20:
            gene_list.append(key)
            gene_groups.append(np.array(temp.loc[((temp["single_gene"]==key) & (temp[y].notna())),y]))
            quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))
quartiles=np.array(quartiles)           
            
if gene_list != []:
        temp=temp.loc[((temp["single_gene"].isin(gene_list)) & (temp[y].notna())),:]
        counts=temp["single_gene"].value_counts()
        counts["all CH"]=gene_groups[1].shape[0]
        
        parts = ax[0].violinplot(gene_groups, showmeans=False, showmedians=False, showextrema=False,widths=0.65,positions=np.arange(0,len(gene_list),1))

        for i,pc in enumerate(parts['bodies']):
            if gene_list[i]=="Control":
                pc.set_facecolor(gene_color_new["Control"])
            else:
                pc.set_facecolor("#999999")
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)

        ax[0].boxplot(gene_groups,positions=np.arange(0,len(gene_list),1),showcaps=False,widths=0.06, patch_artist=True, boxprops=dict(color=black, facecolor=black),whiskerprops=dict(color=black, linewidth=2), medianprops=dict(color="w", linewidth=0 ),showfliers=False)
        inds = np.arange(0, len(quartiles))
        ax[0].scatter(inds, quartiles[:,1], marker='o', color='white', s=30, zorder=3)
        
        xticks_new=[]
        for i in gene_list:
            print(i,counts[i])
            if i != "Control":
                pval_prs=or_table_pr.loc[(or_table_pr[0]==i),4].reset_index()
                pval_prs=pval_prs.iloc[0,1]
            else:
                pval_prs=1
            if pval_prs <0.05:
                xticks_new.append(i[0].upper()+i[1:]+"\n("+str(counts[i])+")\n*")
            else:
                xticks_new.append(i[0].upper()+i[1:]+"\n("+str(counts[i])+")")
        print(xticks_new)
        ax[0].set_xticks(np.arange(0,len(gene_list),1))
        #ax[0].set_xticklabels(xticks_new,rotation=45)
        ax[0].set_xticklabels([])
        ax[0].tick_params(axis='y',labelsize=15,size=7)
        ax[0].tick_params(axis='x',labelsize=12,labelbottom=False)
        ax[0].grid(axis='x')
        ax[0].xaxis.label.set_size(20)
        ax[0].yaxis.label.set_size(22)
        ax[0].set_ylabel("PRS")
        ax[0].set_ylim([-4,4])
        #ax[0].set_xlabel("Gene")
        ax[0].axhline(y=temp.loc[temp["single_gene"]=="Control",y].median(), color='k', linestyle='--')
        print(temp.loc[temp["single_gene"]=="Control",y].median())
                        
temp=data_mca[[y,"MCA_type"]].copy(deep=True)
temp=temp.dropna()
temp["MCA_type"]=temp["MCA_type"].map(lambda row:"autosomal" if row not in ["Y:loss","X:loss","Control"] else row)
y="total_PRS"

temp=temp.loc[(temp[y].notna()),:]
counts=temp["MCA_type"].value_counts()
print(counts.sum())
gene_groups=[]
gene_order=["Control","autosomal","Y:loss","X:loss"]
quartiles=[]
for key in gene_order:
    gene_groups.append(temp.loc[(temp["MCA_type"]==key)& (temp[y].notna()),y])
    quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))
quartiles=np.array(quartiles)           
                       
parts = ax[1].violinplot(gene_groups, showmeans=False, showmedians=False, showextrema=False,widths=0.65,positions=np.arange(0,len(gene_order),1))
mca_color={"Control":gene_color["Control"],"autosomal":"#999999","Y:loss":"#E69F00","X:loss":"#56B4E9"}           
for key in mca_color:
    mca_color[key]=sns.set_hls_values(mca_color[key],s=0.5)
    
for i,pc in enumerate(parts['bodies']):
            if gene_order[i]=="Control":
                pc.set_facecolor(mca_color[gene_order[i]])
            else:
                pc.set_facecolor("#8585ad")
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)

ax[1].boxplot(gene_groups,positions=np.arange(0,len(gene_order),1),showcaps=False,widths=0.06, patch_artist=True,boxprops=dict(color=black, facecolor=black),
            whiskerprops=dict(color=black, linewidth=2),
            medianprops=dict(color="w", linewidth=0 ),showfliers=False)                       
inds = np.arange(0, len(quartiles))
ax[1].scatter(inds, quartiles[:,1], marker='o', color='white', s=30, zorder=3)                       
xticks_new_mca=[]
gene_rename={"autosomal":"autosomal","Y:loss":"LOY","X:loss":"LOX"}
for j,i in enumerate(gene_order):
    print(i,counts[i])
    if i!="Control":
        if plot_values[j-1,3]<0.05:
            xticks_new_mca.append(gene_rename[i]+"\n("+str(counts[i])+")"+"\n*")
        else:
            xticks_new_mca.append(gene_rename[i]+"\n("+str(counts[i])+")")
    else:
        xticks_new_mca.append(i+"\n("+str(counts[i])+")")

ax[1].set_xticks(np.arange(0,len(xticks),1))
ax[1].set_xticklabels([])
ax[1].grid(axis='x')
ax[1].tick_params(axis='x',labelsize=12,labelbottom=False)
ax[1].tick_params(axis='y',labelsize=15)
ax[1].xaxis.label.set_size(25)
ax[1].yaxis.label.set_size(25)
ax[1].set_ylim([-4,4])

ax[1].axhline(y=temp.loc[temp["MCA_type"]=="Control",y].median(), color='k', linestyle='--')
plt.subplots_adjust(wspace=0.05)

#or_table_pr=or_table_pr.set_index(0).loc[[x for x in gene_list if x!="Control"],:]
#or_table_pr=or_table_pr.reset_index()
or_table_pr=or_table_pr.set_index(0).loc[[x for x in gene_list if x!="Control"],:]
or_table_pr=or_table_pr.reset_index()
or_significant=or_table_pr.loc[(or_table_pr[4]>=0.05),:]
ax[2].errorbar(or_significant.index+1,or_significant.loc[:,1],yerr=[or_significant.loc[:,1]-or_significant.loc[:,2],or_significant.loc[:,3]-or_significant.loc[:,1]],fmt="o",markersize=20,elinewidth=2,c="#b3b3b3",label="P"+r'$\mathrm{\geq}0.05$')
or_significant=or_table_pr.loc[(or_table_pr[4]<0.05),:]
ax[2].errorbar(or_significant.index+1,or_significant.loc[:,1],yerr=[or_significant.loc[:,1]-or_significant.loc[:,2],or_significant.loc[:,3]-or_significant.loc[:,1]],fmt="o",markersize=20,elinewidth=2,c="#4d4d4d",label="P"+r'$\mathrm{<}0.05$')

ax[2].set_xticks(np.arange(0,16))
ax[2].set_xticklabels(xticks_new,fontsize=15,rotation=35)
ax[2].set_ylabel("OR",fontsize=22)
ax[2].set_xlabel("CH driver gene",fontsize=22)
ax[2].tick_params(axis='y',labelsize=15,size=7)
ax[2].axhline(y=1,linestyle="--",lw=1)
ax[2].legend()


plot_values_s=plot_values[(plot_values[:,3]<0.05),:]
ax[3].errorbar(np.where(plot_values[:,3]<0.05)[0]+1,plot_values_s[:,0],yerr=[plot_values_s[:,0]-plot_values_s[:,1],plot_values_s[:,2]-plot_values_s[:,0]],fmt="o",markersize=20,elinewidth=2,c="#575782",label="P"+r'$\mathrm{<}0.05$')

plot_values_n=plot_values[plot_values[:,3]>=0.05,:]
ax[3].errorbar(np.where(plot_values[:,3]>=0.05)[0]+1,plot_values_n[:,0],yerr=[plot_values_n[:,0]-plot_values_n[:,1],plot_values_n[:,2]-plot_values_n[:,0]],fmt="o",markersize=20,elinewidth=2,c="#b1b1cb",label="P"+r'$\mathrm{\geq}0.05$')
ax[3].set_xticks(np.arange(0,4))
ax[3].set_xticklabels(xticks_new_mca,fontsize=15,rotation=45)
ax[3].tick_params(axis='y',labelsize=15)
ax[3].axhline(y=1,linestyle="--",lw=1)
ax[3].set_xlabel("mCA type",fontsize=22)
ax[3].legend()

for a in ax:
    a.spines[['right', 'top']].set_visible(False)

#plt.savefig("../figures_revision1/prs_ch_or_mca_singlegene.png",dpi=600,bbox_inches="tight")
#plt.savefig("../figures_revision1/prs_ch_or_mca_singlegene.pdf",bbox_inches="tight")
#plt.savefig("../figures_revision1/prs_ch_or_mca_singlegene.svg",bbox_inches="tight")

plt.show()
            


In [ ]:
supptable_1_gene=or_table_pr.copy(deep=True)
supptable_1_gene.columns=["Gene/mCA","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value"]
supptable_1_gene=supptable_1_gene.sort_values(by="Gene/mCA",key=lambda col: col.str.lower())
supptable_1_gene["FDR"]=multipletests(supptable_1_gene["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

supptable_1=pd.concat([supptable_1_gene[["Gene/mCA","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value","FDR"]],supptable_1_mca[["Gene/mCA","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value","FDR"]]])
supptable_1["FDR_all"]=multipletests(supptable_1["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
#supptable_1.to_csv("prs_coef_supptable_r1.tsv",sep="\t",index=False)

In [ ]:
#supptable_1.to_csv("prs_coef_supptable_singlegene_r1_singlegene.tsv",sep="\t",index=False)

In [ ]:
supptable_1

In [ ]:
############### Association between mCA and measured telomere length ####################
temp=ukb500k_ts_som_mca[["Z-adjusted T/S log","sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins","CHR_COPY_CHANGE","CELL_FRAC","HM"]].copy(deep=True)
temp=temp[~(temp["CHR_COPY_CHANGE"].isin(["Y:neutral","X:neutral","Y:gain","X:gain"]))]
temp.loc[temp["CHR_COPY_CHANGE"]=="Control","CELL_FRAC"]=0
temp=temp.dropna()
temp=temp[temp["HM"]==0]
temp["MCA_type"]=temp["CHR_COPY_CHANGE"].map(lambda row:"autosomal MCA" if row not in ["Y:loss","X:loss","Control"] else row)
y="Z-adjusted T/S log"

counts=temp["MCA_type"].value_counts()
print(counts.sum())
gene_order=["autosomal MCA","Y:loss","X:loss"]
plt.figure(figsize=(6,4))
b=sns.violinplot(data=temp,x="MCA_type",y=y,order=["Control"]+gene_order,linewidth=2,palette={"Control":gene_color["Control"],"autosomal MCA":"#999999","Y:loss":"#E69F00","X:loss":"#56B4E9"})
ax=plt.gca()
xticks=ax.get_xticklabels()
xticks_new=[]
for i in xticks:
    print(i.get_text(),counts[i.get_text()])
    xticks_new.append(i.get_text()+"\n("+str(counts[i.get_text()])+")")
plt.ylim([-5,5])
plt.xticks(np.arange(0,len(xticks),1),xticks_new)
plt.axhline(y=temp.loc[temp["MCA_type"]=="Control",y].median(), color='k', linestyle='--')
plt.grid(axis='x')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.xaxis.label.set_size(18)
ax.yaxis.label.set_size(16)
plt.ylabel("Telomere length")
plt.xlabel("")
#plt.savefig("../figures_new/tl_mca.png",dpi=600,bbox_inches="tight")
#plt.savefig("../figures_new/tl_mca.pdf",bbox_inches="tight")
plt.show()
            


plot_values=[]
for mca_type in ["autosomal MCA","Y:loss","X:loss"]:
    if mca_type=="autosomal MCA":
        data=temp[temp["MCA_type"].isin(["Control",mca_type])].copy(deep=True)
        formula = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins",'C(MCA_type,Treatment(reference="Control"))'])
    elif mca_type=="Y:loss":
        data=temp[(temp["MCA_type"].isin(["Control",mca_type])) & (temp["sex"]=="Male")].copy(deep=True)
        formula = 'tl ~' + ' + '.join(['age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins",'C(MCA_type,Treatment(reference="Control"))'])
    else:
        data=temp[(temp["MCA_type"].isin(["Control",mca_type])) & (temp["sex"]=="Female")].copy(deep=True)
        formula = 'tl ~' + ' + '.join(['age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins",'C(MCA_type,Treatment(reference="Control"))'])
    print(data["MCA_type"].value_counts())
    data=data.rename(columns={"Z-adjusted T/S log":"tl"})
    lin_model=smf.ols(formula=formula,data=data)
    result=lin_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = result.params
    conf = result.conf_int()
    print(result.pvalues)
    key='C(MCA_type, Treatment(reference="Control"))[T.'+mca_type+']'
    plot_values.append([params[key],conf.loc[key,0],conf.loc[key,1],result.pvalues[key]])
    
plot_values_mca=np.array(plot_values)


supptable_2_mca=pd.DataFrame(plot_values_mca)
supptable_2_mca["Gene/mCA"]=["autosomal mCA","LOY","LOX"]
supptable_2_mca.columns=["Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value","Gene/mCA"]
supptable_2_mca=supptable_2_mca[["Gene/mCA","Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value"]]
supptable_2_mca["FDR"]=multipletests(supptable_2_mca["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

plot_values=[]
for mca_type in ["autosomal MCA","Y:loss","X:loss"]:
    if mca_type=="autosomal MCA":
        data=temp[temp["MCA_type"].isin(["Control",mca_type])].copy(deep=True)

        formula = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins",'C(MCA_type,Treatment(reference="Control"))','CELL_FRAC'])
    elif mca_type=="Y:loss":
        data=temp[(temp["MCA_type"].isin(["Control",mca_type])) & (temp["sex"]=="Male")].copy(deep=True)
        formula = 'tl ~' + ' + '.join(['age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins",'C(MCA_type,Treatment(reference="Control"))','CELL_FRAC'])
    else:
        data=temp[(temp["MCA_type"].isin(["Control",mca_type])) & (temp["sex"]=="Female")].copy(deep=True)
        formula = 'tl ~' + ' + '.join(['age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins",'C(MCA_type,Treatment(reference="Control"))','CELL_FRAC'])
    print(data["MCA_type"].value_counts())
    data=data.rename(columns={"Z-adjusted T/S log":"tl"})
    lin_model=smf.ols(formula=formula,data=data)
    result=lin_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = result.params
    conf = result.conf_int()
    print(result.pvalues)
    #key='C(MCA_type, Treatment(reference="Control"))[T.'+mca_type+']'
    key="CELL_FRAC"
    plot_values.append([params[key],conf.loc[key,0],conf.loc[key,1],result.pvalues[key]])
    
plot_values=np.array(plot_values)
data_mca=temp.copy(deep=True)
data_mca=data_mca.rename(columns={"Z-adjusted T/S log":"tl"})

plt.bar(np.arange(3),plot_values_mca[:,0],yerr=[plot_values_mca[:,0]-plot_values_mca[:,1],plot_values_mca[:,2]-plot_values_mca[:,0]],width=0.5)
ticklabel=["autosomal MCA","Y:loss","X:loss"]
ticklabel=[ticklabel[i]+"\n*" if plot_values_mca[i,3]<0.05 else ticklabel[i] for i in range(3)]
plt.xticks(np.arange(3),ticklabel,fontsize=13)
plt.ylabel("Coefficient",fontsize=13)
#plt.yticks([-0.15,-0.10,-0.05])
plt.axhline(y=0,linestyle="--",lw=1)
#plt.savefig("../figures_new/tl_mca_coef.png",dpi=600,bbox_inches="tight")
#plt.savefig("../figures_new/tl_mca_coef.pdf",bbox_inches="tight")

supptable_3_mca=pd.DataFrame(plot_values)
supptable_3_mca["Gene/mCA"]=["autosomal mCA","LOY","LOX"]
supptable_3_mca.columns=["Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value","Gene/mCA"]
supptable_3_mca=supptable_3_mca[["Gene/mCA","Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value"]]
supptable_3_mca["FDR"]=multipletests(supptable_3_mca["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
#supptable_1_mca[["mCA type","Coefficient","Lower CI (2.5%)","Upper CI (97.5%)","P-value"]].to_csv("MCA_cellfrac_coef_supptable.csv",sep="\t",index=False)

In [ ]:
############## Association with telomere length and individual gene+vaf group ##############

data=ukb500k_ts_som[["Z-adjusted T/S log", "sex", "age", "smoking_status", "single_gene", "single_gene_vaf", "single_gene_vaf_0.10", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10", "WBC_wins","MO_perc_wins", "LY_perc_wins", "BA_perc_wins","NE_perc_wins","EO_perc_wins","eid","HM"]].copy(deep=True)
data=data.dropna()
data=data[(data["HM"]==0)]
data["single_gene_vaf_group"]=data["single_gene"]+"_"+data["single_gene_vaf_0.10"].astype("str")
print(data.shape)

data=data.rename(columns={"Z-adjusted T/S log":"tl"})
formula = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','C(single_gene_vaf_group,Treatment(reference="Control_NA"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"])

lin_model_fullreg_gene=smf.ols(formula=formula,data=data)
result_fullreg_gene=lin_model_fullreg_gene.fit(max_iterations= 100000)
display(result_fullreg_gene.summary2())
params_fullreg_gene = result_fullreg_gene.params
conf_fullreg_gene = result_fullreg_gene.conf_int() 

print(result_fullreg_gene.pvalues)


############## Association with telomere length and individual gene ##############

data=ukb500k_ts_som[["Z-adjusted T/S log", "sex", "age", "smoking_status", "single_gene", "single_gene_vaf", "single_gene_vaf_0.10", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10", "WBC_wins","MO_perc_wins", "LY_perc_wins", "BA_perc_wins","NE_perc_wins","EO_perc_wins","eid","HM"]].copy(deep=True)
data=data.dropna()
data=data[(data["HM"]==0)]


print(data.shape)

data=data.rename(columns={"Z-adjusted T/S log":"tl"})
formula = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','C(single_gene,Treatment(reference="Control"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"])

lin_model=smf.ols(formula=formula,data=data)
result_fullreg_gene=lin_model.fit(max_iterations= 100000)
display(result_fullreg_gene.summary2())
params_fullreg_gene = result_fullreg_gene.params
conf_fullreg_gene = result_fullreg_gene.conf_int()

print(result_fullreg_gene.pvalues)



############## Association with telomere length and all CH ################

data=ukb500k_ts_som[["Z-adjusted T/S log", "sex", "age", "smoking_status", "single_gene", "single_gene_vaf", "single_gene_vaf_0.10", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10", "WBC_wins","MO_perc_wins", "LY_perc_wins", "BA_perc_wins","NE_perc_wins","EO_perc_wins","eid","HM"]].copy(deep=True)
data=data.dropna()
data=data[(data["HM"]==0)]

print(data.shape)

data=data.rename(columns={"Z-adjusted T/S log":"tl"})
data["single_gene"]=data["single_gene"].map(lambda row:row if row=="Control" else "all CH")
formula = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','C(single_gene,Treatment(reference="Control"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"])

lin_model=smf.ols(formula=formula,data=data)
result_fullreg_allch=lin_model.fit(max_iterations= 100000)
display(result_fullreg_allch.summary2())
params_fullreg_allch = result_fullreg_allch.params
conf_fullreg_allch = result_fullreg_allch.conf_int()

print(result_fullreg_allch.pvalues)


data["single_gene"].value_counts()

############## Association with telomere length and all CH vaf ################

data=ukb500k_ts_som[["Z-adjusted T/S log", "sex", "age", "smoking_status", "single_gene", "single_gene_vaf", "single_gene_vaf_0.10", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10", "WBC_wins","MO_perc_wins", "LY_perc_wins", "BA_perc_wins","NE_perc_wins","EO_perc_wins","eid","HM"]].copy(deep=True)
data=data.dropna()
data=data[(data["HM"]==0)]

print(data.shape)

data=data.rename(columns={"Z-adjusted T/S log":"tl"})
data["single_gene"]=data["single_gene"].map(lambda row:row if row=="Control" else "all CH")
data.loc[(data["single_gene_vaf"]=="NA"),"single_gene_vaf"]=0
data["single_gene_vaf"]=data["single_gene_vaf"].astype(float)

formula = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','C(single_gene,Treatment(reference="Control"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins","single_gene_vaf"])

lin_model=smf.ols(formula=formula,data=data)
result_fullreg_allch_vaf=lin_model.fit(max_iterations= 100000)
display(result_fullreg_allch_vaf.summary2())
params_allch_vaf = result_fullreg_allch_vaf.params
conf_fullreg_allch_vaf = result_fullreg_allch_vaf.conf_int()

print(result_fullreg_allch_vaf.pvalues)


############## Association with telomere length and individual gene CH vaf ################

data=ukb500k_ts_som[["Z-adjusted T/S log", "sex", "age", "smoking_status", "single_gene", "single_gene_vaf", "single_gene_vaf_0.10", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10", "WBC_wins", "MO_perc_wins", "LY_perc_wins", "BA_perc_wins","NE_perc_wins","EO_perc_wins","eid","HM"]].copy(deep=True)
data=data.dropna()
data=data[(data["HM"]==0)]

print(data.shape)

gene_vaf=[]
for gene in data.single_gene.unique():
    if gene != "Control":
        data[gene+"_vaf"]=data.apply(lambda row:row["single_gene_vaf"] if row["single_gene"]==gene else 0,axis=1)
        gene_vaf.append(gene+"_vaf")

data=data.rename(columns={"Z-adjusted T/S log":"tl"})
formula = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','C(single_gene,Treatment(reference="Control"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"]+gene_vaf)

lin_model=smf.ols(formula=formula,data=data)
result_fullreg=lin_model.fit(max_iterations= 100000)
display(result_fullreg.summary2())
params_fullreg = result_fullreg.params
conf_fullreg = result_fullreg.conf_int()

print(result_fullreg.pvalues)




supptable_2_gene=pd.DataFrame(pd.concat([params_fullreg_gene,conf_fullreg_gene,result_fullreg_gene.pvalues],axis=1))
supptable_2_gene=(supptable_2_gene.reset_index().iloc[4:18,:]).reset_index(drop=True)
supptable_2_gene=supptable_2_gene.append((pd.concat([params_fullreg_allch,conf_fullreg_allch,result_fullreg_allch.pvalues],axis=1).loc['C(single_gene, Treatment(reference="Control"))[T.all CH]',:]).to_frame().T.reset_index())
display(supptable_2_gene)
supptable_2_gene["Gene/mCA"]=supptable_2_gene["index"].map(lambda row:row.split("[")[-1].strip("]")[2:])
supptable_2_gene.columns=["Index","Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value","Gene/mCA"]
supptable_2_gene=supptable_2_gene.sort_values(by="Gene/mCA",key=lambda col: col.str.lower())
supptable_2_gene["FDR"]=multipletests(supptable_2_gene["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

supptable_2=pd.concat([supptable_2_gene[["Gene/mCA","Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value","FDR","FDR_selected"]],supptable_2_mca[["Gene/mCA","Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value","FDR"]]])
supptable_2["FDR_all"]=multipletests(supptable_2["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

#supptable_2.to_csv("coef_supptable_r1.tsv",sep="\t",index=False)

#supptable_2.to_csv("coef_supptable_singlegene_r1_n.tsv",sep="\t",index=False)

supptable_2

supptable_3_gene=pd.DataFrame(pd.concat([params_fullreg,conf_fullreg,result_fullreg.pvalues],axis=1))
supptable_3_gene=(supptable_3_gene.reset_index().iloc[35:49,:]).reset_index(drop=True)
supptable_3_gene=supptable_3_gene.append((pd.concat([params_allch_vaf,conf_fullreg_allch_vaf,result_fullreg_allch_vaf.pvalues],axis=1).loc['single_gene_vaf',:]).to_frame().T.reset_index())
supptable_3_gene["Gene/mCA"]=supptable_3_gene["index"].map(lambda row:row[:-4])
supptable_3_gene.columns=["Index","Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value","Gene/mCA"]
supptable_3_gene=supptable_3_gene.sort_values(by="Gene/mCA",key=lambda col: col.str.lower())
supptable_3_gene["FDR"]=multipletests(supptable_3_gene["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
supptable_3_gene.loc[(supptable_3_gene["Gene/mCA"].isin(["ASXL1","CBL","GNAS", "GNB1","PPM1D","SF3B1","SRSF2","TP53","U2AF1"])),"FDR_selected"]=multipletests(supptable_3_gene.loc[(supptable_3_gene["Gene/mCA"].isin(["ASXL1","CBL","GNAS", "GNB1","PPM1D","SF3B1","SRSF2","TP53","U2AF1"])),"P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

supptable_3=pd.concat([supptable_3_gene[["Gene/mCA","Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value","FDR","FDR_selected"]],supptable_3_mca[["Gene/mCA","Coefficient","Coefficient lower CI (2.5%)","Coefficient upper CI (97.5%)","P-value","FDR"]]])
supptable_3["FDR_all"]=multipletests(supptable_3["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

#supptable_3.to_csv("vaf_coef_supptable_r1.tsv",sep="\t",index=False)
display(supptable_3)


supptable_3_gene

#supptable_3.to_csv("vaf_coef_supptable_singlegene_r1_check.tsv",sep="\t",index=False)


supptable_3



"""
left_0, width_0 = 0.04, 14*0.05
left_1, width_1 = left_0+width_0+0.02,4*0.05


bottom_2, height_2 = 0.1,0.076*1.5
gap_0_1=0.01
bottom_1, height_1 = bottom_2+height_2+gap_0_1,0.076*5
gap_1_2=0.03
bottom_0, height_0 = bottom_1+height_1+gap_1_2,0.076*6

plt.figure(figsize=(26,12))
axes=np.array([np.array([plt.axes([left_0, bottom_0, width_0, height_0]),plt.axes([left_1, bottom_0, width_1, height_0])]),np.array([plt.axes([left_0, bottom_1, width_0, height_1]),plt.axes([left_1, bottom_1, width_1, height_1])]),np.array([plt.axes([left_0, bottom_2, width_0, height_2]),plt.axes([left_1, bottom_2, width_1, height_2])])])

"""

fig, axes = plt.subplots(3, 2,sharex=False,sharey=False,figsize=(28,12),gridspec_kw={'height_ratios':[5,5,1.5],'width_ratios':[15,4]})
y="tl"
temp=data[["single_gene",y]].copy(deep=True)
gene_count=Counter(temp["single_gene"])

gene_color_new={}
for key in gene_color:
    gene_color_new[key]=sns.set_hls_values(gene_color[key],s=0.5)
black=sns.set_hls_values('#000000',s=0.5)

gene_list=[]
gene_groups=[]
quartiles=[]

gene_list.append("Control")
gene_groups.append(np.array(temp.loc[((temp["single_gene"]=="Control") & (temp[y].notna())),y]))
quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))

gene_list.append("all CH")
gene_groups.append(np.array(temp.loc[((temp["single_gene"].isin(["ASXL1","CALR","CBL","DNMT3A_R882","DNMT3A_other","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1"])) & (temp[y].notna())),y]))
quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))

for key in ["ASXL1","CALR","CBL","DNMT3A_R882","DNMT3A_other","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1"]:
        if gene_count[key]>=20:
            gene_list.append(key)
            gene_groups.append(np.array(temp.loc[((temp["single_gene"]==key) & (temp[y].notna())),y]))
            quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))
            print(gene_groups[-1].shape,gene_count[key])
quartiles=np.array(quartiles)           
            
if gene_list != []:
        temp=temp.loc[((temp["single_gene"].isin(gene_list)) & (temp[y].notna())),:]
        counts=temp["single_gene"].value_counts()
        
        parts = axes[0,0].violinplot(gene_groups, showmeans=False, showmedians=False, showextrema=False,widths=0.9,positions=np.arange(-2,2*len(gene_list)-2,2))

        for i,pc in enumerate(parts['bodies']):
            if gene_list[i]=="Control":
                pc.set_facecolor(gene_color_new[gene_list[i]])
            else:
                pc.set_facecolor("#999999")
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)

        axes[0,0].boxplot(gene_groups,positions=np.arange(-2,2*len(gene_list)-2,2),showcaps=False,widths=0.12, patch_artist=True, boxprops=dict(color=black, facecolor=black),whiskerprops=dict(color=black, linewidth=2), medianprops=dict(color="w", linewidth=0 ),showfliers=False)
        inds = np.arange(0, len(quartiles))
        axes[0,0].scatter(np.arange(-2,2*len(gene_list)-2,2), quartiles[:,1], marker='o', color='white', s=30, zorder=3)
        
        axes[0,0].tick_params(axis='y',labelsize=15)
        axes[0,0].tick_params(axis='x',labelsize=15)
        axes[0,0].grid(axis='x')
        axes[0,0].xaxis.label.set_size(20)
        axes[0,0].yaxis.label.set_size(20)
        axes[0,0].set_ylabel("LTL")
        axes[0,0].set_xlabel("CH driver gene",labelpad=8)
        axes[0,0].axhline(y=temp.loc[temp["single_gene"]=="Control",y].median(), color='k', linestyle='--')
        axes[0,0].set_ylim([-5,5])
        axes[0,0].xaxis.set_tick_params(which='both', labelbottom=True)
        print(temp.loc[temp["single_gene"]=="Control",y].median())  
        ticklabels=["Control"]+["All CH\n*" if result_fullreg_allch.pvalues['C(single_gene, Treatment(reference="Control"))[T.all CH]']<0.05 else "All CH"]+[x+"\n*" if result_fullreg_gene.pvalues['C(single_gene, Treatment(reference="Control"))[T.'+x+']']<0.05 else x for x in gene_list[2:]]
        print(ticklabels)
        axes[0,0].set_xticks(np.arange(-2,30,2))
        axes[0,0].set_xticklabels(ticklabels,rotation=15)
        axes[0,0].spines[['right', 'top']].set_visible(False)
                       
temp=data_mca[[y,"MCA_type","CELL_FRAC"]].copy(deep=True)
temp=temp.loc[(temp[y].notna()),:]
temp["MCA_type"]=temp["MCA_type"].map(lambda row:"autosomal" if row not in ["Y:loss","X:loss","Control"] else row)
counts=temp["MCA_type"].value_counts()
print(counts)
gene_groups=[]
gene_order=["Control","autosomal","Y:loss","X:loss"]
quartiles=[]
for key in gene_order:
    gene_groups.append(np.array(temp.loc[(temp["MCA_type"]==key)& (temp[y].notna()),y]))
    quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))
quartiles=np.array(quartiles)           
                       
parts = axes[0,1].violinplot(gene_groups, showmeans=False, showmedians=False, showextrema=False,widths=0.9,positions=np.arange(-2,6,2))
mca_color={"Control":gene_color["Control"],"autosomal":"#999999","Y:loss":"#E69F00","X:loss":"#56B4E9"}           
for key in mca_color:
    mca_color[key]=sns.set_hls_values(mca_color[key],s=0.5)
    
for i,pc in enumerate(parts['bodies']):
            if gene_order[i]=="Control":
                pc.set_facecolor(mca_color[gene_order[i]])
            else:
                pc.set_facecolor("#8585ad")
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)

axes[0,1].boxplot(gene_groups,positions=np.arange(-2,6,2),showcaps=False,widths=0.12, patch_artist=True,boxprops=dict(color=black, facecolor=black),
            whiskerprops=dict(color=black, linewidth=2),
            medianprops=dict(color="w", linewidth=0 ),showfliers=False)                       
inds = np.arange(0, len(quartiles))
axes[0,1].scatter(np.arange(-2,6,2), quartiles[:,1], marker='o', color='white', s=30, zorder=3)

axes[0,1].set_xticks(np.arange(-2,6,2))
axes[0,1].set_xticklabels(gene_order,rotation=35)
axes[0,1].spines[['right', 'top']].set_visible(False)
#axes[0,1].set_yticklabels([])
axes[0,1].grid(axis='x')
axes[0,1].tick_params(axis='x',labelsize=15)
axes[0,1].tick_params(axis='y',labelsize=15)
axes[0,1].xaxis.label.set_size(20)
axes[0,1].yaxis.label.set_size(20)
axes[0,1].set_xlabel("mCA type",labelpad=20)
axes[0,1].set_ylim([-5,5])
axes[0,1].axhline(y=temp.loc[temp["MCA_type"]=="Control",y].median(), color='k', linestyle='--')
axes[0,1].xaxis.set_tick_params(which='both', labelbottom=True)
ticklabels=["Control"]+[x+"\n*" if plot_values_mca[i,3]<0.05 else x for i,x in enumerate(["autosomal","LOY","LOX"])]
axes[0,1].set_xticklabels(ticklabels,rotation=15)

###################################################################################################


y="tl"#"Z-adjusted T/S log"
temp=data[["single_gene","single_gene_vaf","single_gene_vaf_0.10",y]].copy(deep=True)
temp=temp[(temp["single_gene"]!="NA") & (temp[y].notna())]
gene_order_plot=["all CH", "ASXL1", "CALR", "CBL", "DNMT3A_R882", "DNMT3A_other", "GNAS", "GNB1", "JAK2", "PPM1D", "SF3B1","SRSF2", "TET2","TP53","U2AF1"]
black=sns.set_hls_values('#000000',s=0.5)


for vaf in ["0.10"]:
        gene_vaf_groups=temp.groupby(["single_gene","single_gene_vaf_0.10"])
        gene_vaf_groups_dict={}
        
        gene_vaf_groups_dict["all CH_0"]=np.array(temp.loc[((temp["single_gene"].isin(gene_order_plot)) & (temp["single_gene_vaf_0.10"]==0)),y].dropna())
        gene_vaf_groups_dict["all CH_1"]=np.array(temp.loc[((temp["single_gene"].isin(gene_order_plot)) & (temp["single_gene_vaf_0.10"]==1)),y].dropna())
        
        for group_name, df_group in gene_vaf_groups:
            gene_vaf_groups_dict[group_name[0]+"_"+str(group_name[1])]=np.array(df_group[y].dropna())
        
        gene_vaf_group_list=[gene_vaf_groups_dict["Control_NA"]]
        pos=[-2]
        colors_violin=[sns.set_hls_values(gene_color["Control"],s=0.5)]
        percentiles=[np.percentile(gene_vaf_groups_dict["Control_NA"],[25, 50, 75])]
        
        for i,gene in enumerate(gene_order_plot):
            gene_vaf_group_list.append(gene_vaf_groups_dict[gene+"_0"])
            gene_vaf_group_list.append(gene_vaf_groups_dict[gene+"_1"])
            pos+=[i*2-0.45,i*2+0.45]
            #colors_violin+=[sns.set_hls_values('#70db70',s=0.5),sns.set_hls_values("#ff99e6",s=0.5)]
            colors_violin+=['#737373',"#bfbfbf"]#808080
            percentiles.append(np.percentile(gene_vaf_groups_dict[gene+"_0"],[25, 50, 75]))
            percentiles.append(np.percentile(gene_vaf_groups_dict[gene+"_1"],[25, 50, 75]))
        percentiles=np.array(percentiles)
        axes[1,0].set_ylim([-5,5])
        counts=temp[["single_gene","single_gene_vaf_"+str(vaf)]].value_counts()
        counts.loc["all CH",0]=gene_vaf_groups_dict["all CH_0"].shape[0]
        counts.loc["all CH",1]=gene_vaf_groups_dict["all CH_1"].shape[0]
        display (counts)
        parts=axes[1,0].violinplot(gene_vaf_group_list, pos, widths=0.6,
                     showmedians=False,showextrema=False)
        for i,pc in enumerate(parts['bodies']):
            pc.set_facecolor(colors_violin[i])
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)
        axes[1,0].boxplot(gene_vaf_group_list,positions=pos,showcaps=False,widths=0.08, patch_artist=True, boxprops=dict(color=black, facecolor=black),whiskerprops=dict(color=black, linewidth=2), medianprops=dict(color="w", linewidth=0 ),showfliers=False)
        
        quartile1, medians, quartile3 = percentiles[:,0],percentiles[:,1],percentiles[:,2]

        axes[1,0].scatter(pos, medians, marker='o', color='white', s=25, zorder=3)
        
        xticks_new=["Control\n("+str(counts["Control","NA"])+")"]
        result_fullreg_pvalues=result_fullreg.pvalues
        result_fullreg_pvalues["all CH_vaf"]=result_fullreg_allch_vaf.pvalues["single_gene_vaf"]
        for i in gene_order_plot:
            print(i,counts[i])
            try:
                low_vaf=str(counts[i,0])
            except KeyError:
                low_vaf="0"
            try:
                high_vaf=str(counts[i,1])
            except KeyError:
                high_vaf="0"
            if result_fullreg_pvalues[i+"_vaf"]<0.05:
                xticks_new.append(i[0].upper()+i[1:]+"\n("+low_vaf+","+high_vaf+")\n*")
            else:
                xticks_new.append(i[0].upper()+i[1:]+"\n("+low_vaf+","+high_vaf+")")

        print(xticks_new)
        
        axes[1,0].axhline(y=temp.loc[temp["single_gene"]=="Control",y].median(), color='k', linestyle='--')
        axes[1,0].set_ylabel("LTL",fontsize=20)
        axes[1,0].set_xlabel("")
        axes[1,0].set_xticks([])
        axes[1,0].spines[['right', 'top']].set_visible(False)
        axes[1,0].tick_params(labelsize=15)
        legend_handles = [mpatches.Patch(color=sns.set_hls_values(gene_color["Control"],s=0.5), label='Control'),mpatches.Patch(color='#737373',label="VAF "+r'$\mathrm{<} $'+vaf),mpatches.Patch(color='#bfbfbf',label="VAF "+r'$\mathrm{\geq} $'+vaf)]
        axes[1,0].legend(handles=legend_handles,fontsize=15,loc="upper left")
        for i in range(1,len(pos),2): 
            if result_fullreg_pvalues[gene_order_plot[int((i-1)/2)]+"_vaf"]<0.05:
                if medians[i+1]-medians[i]>=0:
                    axes[1,0].arrow(pos[i],medians[i],pos[i+1]-pos[i],medians[i+1]-medians[i],width=0.05,length_includes_head=True,head_width=0.45,head_length=0.4,linestyle=(0, (5,10)),color='red')
                else:
                    axes[1,0].arrow(pos[i],medians[i],pos[i+1]-pos[i],medians[i+1]-medians[i],width=0.05,length_includes_head=True,head_width=0.45,head_length=0.4,linestyle=(0, (5,10)),color='b')
        #axes[1,0].arrow(pos[1],medians[1],pos[2]-pos[1],medians[2]-medians[1],width=0.05,length_includes_head=True,head_width=0.45,head_length=0.4,linestyle=(0, (5,10)),color='b')
                
gene_vaf_order=[x+"_vaf" for x in gene_order_plot]
params_fullreg["all CH_vaf"]=result_fullreg_allch_vaf.params["single_gene_vaf"]
conf_fullreg.loc["all CH_vaf"]=conf_fullreg_allch_vaf.loc["single_gene_vaf"]

axes[2,0].bar(np.arange(0,30,2),params_fullreg[gene_vaf_order],yerr=[params_fullreg[gene_vaf_order]-conf_fullreg.loc[gene_vaf_order,0],conf_fullreg.loc[gene_vaf_order,1]-params_fullreg[gene_vaf_order]],width=1.2)
axes[2,0].set_ylabel("Coefficient",fontsize=20)
axes[2,0].spines[['right', 'top']].set_visible(False)
axes[2,0].axhline(y=0, color='k', linestyle='--', lw=1)
axes[2,0].tick_params(labelsize=15)
axes[2,0].set_xticks(np.arange(-2,30,2))
print(xticks_new)
axes[2,0].set_xticklabels(xticks_new,fontsize=15,rotation=35)


#gene_order_plot=[x+"\n*" if result_fullreg.pvalues[x+"_vaf"]<0.05 else x for x in gene_order_plot ]
#axes[2,0].set_xticklabels(["Control"]+gene_order_plot,fontsize=15,rotation=35)

print(["Control"]+gene_order_plot)
plt.subplots_adjust(hspace=0.05)
plt.subplots_adjust(wspace=0.05)

y="tl"
temp=data_mca[[y,"MCA_type","CELL_FRAC"]].copy(deep=True)
temp["mca_0.2"]=temp["CELL_FRAC"].map(lambda row:"1" if row>=0.2 else "0",na_action='ignore')
temp["MCA_type"]=temp["MCA_type"].map(lambda row:"autosomal" if row not in ["Y:loss","X:loss","Control"] else row)
temp=temp.loc[(temp[y].notna()),:]
print(temp[["mca_0.2","MCA_type"]].value_counts())
temp.loc[temp["MCA_type"]=="Control","mca_0.2"]="NA"

gene_vaf_groups=temp.groupby(["MCA_type","mca_0.2"])
gene_vaf_groups_dict={}
for group_name, df_group in gene_vaf_groups:
    gene_vaf_groups_dict[group_name[0]+"_"+str(group_name[1])]=np.array(df_group[y].dropna())
    
gene_vaf_group_list=[gene_vaf_groups_dict["Control_NA"]]
pos=[-2]
colors_violin=[sns.set_hls_values(gene_color["Control"],s=0.5)]
percentiles=[np.percentile(gene_vaf_groups_dict["Control_NA"],[25, 50, 75])]

for i,gene in enumerate(["autosomal","Y:loss","X:loss"]):
    gene_vaf_group_list.append(gene_vaf_groups_dict[gene+"_0"])
    gene_vaf_group_list.append(gene_vaf_groups_dict[gene+"_1"])
    pos+=[i*2-0.45,i*2+0.45]
    colors_violin+=['#5c5c8a',"#b3b3cc"]
    percentiles.append(np.percentile(gene_vaf_groups_dict[gene+"_0"],[25, 50, 75]))
    percentiles.append(np.percentile(gene_vaf_groups_dict[gene+"_1"],[25, 50, 75]))
percentiles=np.array(percentiles)
      
axes[1,1].set_ylim([-5,5])
counts=temp[["MCA_type","mca_0.2"]].value_counts()
display (counts)
parts=axes[1,1].violinplot(gene_vaf_group_list, pos, widths=0.6,
                     showmedians=False,showextrema=False)
for i,pc in enumerate(parts['bodies']):
    pc.set_facecolor(colors_violin[i])
    pc.set_edgecolor(black)
    pc.set_alpha(1)
    pc.set_linewidth(1.5)
axes[1,1].boxplot(gene_vaf_group_list,positions=pos,showcaps=False,widths=0.08, patch_artist=True, boxprops=dict(color=black, facecolor=black),whiskerprops=dict(color=black, linewidth=2), medianprops=dict(color="w", linewidth=0 ),showfliers=False)
        
quartile1, medians, quartile3 = percentiles[:,0],percentiles[:,1],percentiles[:,2]

axes[1,1].scatter(pos, medians, marker='o', color='white', s=25, zorder=3)

xticks_new=["Control\n("+str(counts["Control","NA"])+")"]
gene_rename={"autosomal":"autosomal","Y:loss":"LOY","X:loss":"LOX"}
for i,gene in enumerate(["autosomal","Y:loss","X:loss"]):
    try:
        low_vaf=str(counts[gene,"0"])
    except KeyError:
        low_vaf="0"
    try:
        high_vaf=str(counts[gene,"1"])
    except KeyError:
        high_vaf="0"
    if plot_values[i,3]<0.05:
        xticks_new.append(gene_rename[gene]+"\n("+low_vaf+","+high_vaf+")\n*")
    else:
        xticks_new.append(gene_rename[gene]+"\n("+low_vaf+","+high_vaf+")")
        
axes[1,1].axhline(y=temp.loc[temp["MCA_type"]=="Control",y].median(), color='k', linestyle='--')
        

axes[1,1].set_xlabel("")
axes[1,1].set_xticks([])
legend_handles = [mpatches.Patch(color=sns.set_hls_values(gene_color["Control"],s=0.5),label='Control'),mpatches.Patch(color='#5c5c8a',label="Cell fraction"+r'$\mathrm{<}$'+" 0.2"),mpatches.Patch(color="#bfbfbf",label="Cell fraction"+r'$\mathrm{\geq}$'+" 0.2")]
axes[1,1].legend(handles=legend_handles,fontsize=15)
for i in range(1,len(pos),2):
            if plot_values[int((i-1)/2),3]<0.05:
                if medians[i+1]-medians[i]>=0:
                    axes[1,1].arrow(pos[i],medians[i],pos[i+1]-pos[i],medians[i+1]-medians[i],width=0.05,length_includes_head=True,head_width=0.45,head_length=0.4,linestyle=(0, (5,10)),color='red')
                else:
                    axes[1,1].arrow(pos[i],medians[i],pos[i+1]-pos[i],medians[i+1]-medians[i],width=0.05,length_includes_head=True,head_width=0.45,head_length=0.4,linestyle=(0, (5,10)),color='b')


axes[1,1].spines[['right', 'top']].set_visible(False)
axes[1,1].tick_params(axis='y',labelsize=15)


axes[2,1].bar(np.arange(0,6,2),plot_values[:,0],yerr=[plot_values[:,0]-plot_values[:,1],plot_values[:,2]-plot_values[:,0]],width=1.2)
ticklabel=["autosomal","Y:loss","X:loss"]
ticklabel=["Control"]+[ticklabel[i]+"\n*" if plot_values[i,3]<0.05 else ticklabel[i] for i in range(3)]
axes[2,1].set_xticks(np.arange(-2,6,2))
axes[2,1].set_xticklabels(xticks_new,rotation=45)
#plt.yticks([-0.15,-0.10,-0.05])
axes[2,1].axhline(y=0,linestyle="--",lw=1)
axes[2,1].tick_params('both',labelsize=15)
axes[2,1].spines[['right', 'top']].set_visible(False)


axes[0,0].set_xlim([-2.5, 28.95])
axes[1,0].set_xlim([-2.5, 28.95])
axes[2,0].set_xlim([-2.5, 28.95])
axes[0,1].set_xlim([-2.5, 4.95])
axes[1,1].set_xlim([-2.5, 4.95])
axes[2,1].set_xlim([-2.5, 4.95])
plt.subplots_adjust(wspace=0.05,hspace=0.1)
"""
plt.savefig("../figures_revision1/tl_ch_mca_singlegene_noHM.png",dpi=600,bbox_inches="tight")
plt.savefig("../figures_revision1/tl_ch_mca_singlegene_noHM.pdf",bbox_inches="tight")
plt.savefig("../figures_revision1/tl_ch_mca_singlegene_noHM.svg",bbox_inches="tight")
plt.show()
"""

In [ ]:
#Get all relevant data (OR, frequencies, pvalues and counts)
#Include VAF<0.1
######## Figure 2C##############

n=2
freq_all,prs_freq_all,tl_freq_all,contigency_table_all,prs_counts,tl_counts,or_all,pv_all,prs_or_all,prs_pv_all,tl_or_all,tl_pv_all=[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)],[[] for i in range(n)]

for i,n in enumerate([2]):
    temp=ukb500k_ts_som.loc[:,["total_PRS","Z-adjusted T/S log","single_gene", "single_gene_vaf","HM"] ].copy(deep=True)
    temp=temp[(temp["total_PRS"].notna()) & (temp["Z-adjusted T/S log"].notna()) & (temp["HM"]==0)]
    temp["prs_q"]=pd.qcut(temp["total_PRS"],q = n, labels = False)
    temp["tl_q"]=pd.qcut(temp["Z-adjusted T/S log"],q = n, labels = False)
    ref_freq=pd.crosstab(temp["prs_q"],temp["tl_q"])
    freq_row,prs_freq_row,tl_freq_row,contigency_table_row,prs_counts_row,tl_counts_row,or_row,pv_row=[],[],[],[],[],[],[],[]
    for j,gene in enumerate(["DNMT3A_other","TET2","SRSF2","PPM1D"]):
        temp_1=temp.loc[(temp["single_gene"]==gene),["prs_q","tl_q","single_gene_vaf"]]
        temp_1=temp_1[(temp_1["single_gene_vaf"].astype(float)<0.1)]
        s=temp_1.shape[0]
        #print(pd.crosstab(temp_1["prs_q"],temp_1["tl_q"])*temp.shape[0],(ref_freq*s))
        freq=pd.crosstab(temp_1["prs_q"],temp_1["tl_q"])*temp.shape[0]/(ref_freq*s)
        freq_all[i].append(freq)
        prs_freq_all[i].append(temp_1["prs_q"].value_counts().reindex(np.arange(n))*temp.shape[0]/(temp["prs_q"].value_counts().reindex(np.arange(n))*s))
        tl_freq_all[i].append(temp_1["tl_q"].value_counts().reindex(np.arange(n))*temp.shape[0]/(temp["tl_q"].value_counts().reindex(np.arange(n))*s))
        contigency_table_all[i].append(pd.crosstab(temp_1["prs_q"],temp_1["tl_q"]))
        prs_counts[i].append(temp_1["prs_q"].value_counts().reindex(np.arange(n)))
        tl_counts[i].append(temp_1["tl_q"].value_counts().reindex(np.arange(n)))
        or_table_all,pvalue_table_all=[[] for x in range(n)],[[] for x in range(n)]
        or_table_prs,pvalue_table_prs,or_table_tl,pvalue_table_tl=[],[],[],[]
        
        temp_1=temp.copy(deep=True)
        temp_1[gene]=0
        temp_1.loc[(temp_1["single_gene_vaf"]=="NA"),"single_gene_vaf"]=0
        temp_1.loc[(temp_1["single_gene"]==gene) & (temp_1["single_gene_vaf"].astype(float)<0.1),gene]=1
        for y in range(n):
            for z in range(n):
                temp_1["prs_tl"]=0
                temp_1.loc[(temp_1["prs_q"]==y) & (temp_1["tl_q"]==z),"prs_tl"]=1
                fe_or,fe_pv=fisher_exact(pd.crosstab(temp_1[gene],temp_1["prs_tl"]))
                or_table_all[y].append(fe_or)
                pvalue_table_all[y].append(fe_pv)
                
        or_all[i].append(np.array([np.array(row,dtype=float) for row in or_table_all],dtype=float))
        pv_all[i].append(np.array([np.array(row,dtype=float) for row in pvalue_table_all],dtype=float))
                
        for y in range(n):
            temp_1["prs_group"]=0
            temp_1.loc[(temp_1["prs_q"]==y),"prs_group"]=1
            #print(pd.crosstab(temp_1[gene],temp_1["prs_group"]))
            fe_or,fe_pv=fisher_exact(pd.crosstab(temp_1[gene],temp_1["prs_group"]))
            or_table_prs.append(fe_or)
            pvalue_table_prs.append(fe_pv)
            
            temp_1["tl_group"]=0
            temp_1.loc[(temp_1["tl_q"]==y),"tl_group"]=1
            fe_or,fe_pv=fisher_exact(pd.crosstab(temp_1[gene],temp_1["tl_group"]))
            or_table_tl.append(fe_or)
            pvalue_table_tl.append(fe_pv)
        
        prs_or_all[i].append(np.array(or_table_prs))
        prs_pv_all[i].append(np.array(pvalue_table_prs))
        tl_or_all[i].append(np.array(or_table_tl))
        tl_pv_all[i].append(np.array(pvalue_table_tl))
        
        
    i=1
    temp=ukb500k_ts_som.loc[:,["total_PRS","Z-adjusted T/S log","single_gene", "single_gene_vaf"] ].copy(deep=True)
    temp=temp[(temp["total_PRS"].notna()) & (temp["Z-adjusted T/S log"].notna())]
    temp["prs_q"]=pd.qcut(temp["total_PRS"],q = n, labels = False)
    temp["tl_q"]=pd.qcut(temp["Z-adjusted T/S log"],q = n, labels = False)
    ref_freq=pd.crosstab(temp["prs_q"],temp["tl_q"])
    freq_row,prs_freq_row,tl_freq_row,contigency_table_row,prs_counts_row,tl_counts_row,or_row,pv_row=[],[],[],[],[],[],[],[]
    for j,gene in enumerate(["DNMT3A_other","TET2","SRSF2","PPM1D"]):
        temp_1=temp.loc[(temp["single_gene"]==gene),["prs_q","tl_q","single_gene_vaf"]]
        temp_1=temp_1[(temp_1["single_gene_vaf"].astype(float)>=0.1)]
        s=temp_1.shape[0]
        #print(pd.crosstab(temp_1["prs_q"],temp_1["tl_q"])*temp.shape[0],(ref_freq*s))
        freq=pd.crosstab(temp_1["prs_q"],temp_1["tl_q"])*temp.shape[0]/(ref_freq*s)
        freq_all[i].append(freq)
        prs_freq_all[i].append(temp_1["prs_q"].value_counts().reindex(np.arange(n))*temp.shape[0]/(temp["prs_q"].value_counts().reindex(np.arange(n))*s))
        tl_freq_all[i].append(temp_1["tl_q"].value_counts().reindex(np.arange(n))*temp.shape[0]/(temp["tl_q"].value_counts().reindex(np.arange(n))*s))
        contigency_table_all[i].append(pd.crosstab(temp_1["prs_q"],temp_1["tl_q"]))
        prs_counts[i].append(temp_1["prs_q"].value_counts().reindex(np.arange(n)))
        tl_counts[i].append(temp_1["tl_q"].value_counts().reindex(np.arange(n)))
        or_table_all,pvalue_table_all=[[] for x in range(n)],[[] for x in range(n)]
        or_table_prs,pvalue_table_prs,or_table_tl,pvalue_table_tl=[],[],[],[]
        
        temp_1=temp.copy(deep=True)
        temp_1[gene]=0
        temp_1.loc[(temp_1["single_gene_vaf"]=="NA"),"single_gene_vaf"]=0
        temp_1.loc[(temp_1["single_gene"]==gene) & (temp_1["single_gene_vaf"].astype(float)>=0.1),gene]=1
        for y in range(n):
            for z in range(n):
                temp_1["prs_tl"]=0
                temp_1.loc[(temp_1["prs_q"]==y) & (temp_1["tl_q"]==z),"prs_tl"]=1
                fe_or,fe_pv=fisher_exact(pd.crosstab(temp_1[gene],temp_1["prs_tl"]))
                or_table_all[y].append(fe_or)
                pvalue_table_all[y].append(fe_pv)
                
        or_all[i].append(np.array([np.array(row,dtype=float) for row in or_table_all],dtype=float))
        pv_all[i].append(np.array([np.array(row,dtype=float) for row in pvalue_table_all],dtype=float))
                
        for y in range(n):
            temp_1["prs_group"]=0
            temp_1.loc[(temp_1["prs_q"]==y),"prs_group"]=1
            #print(pd.crosstab(temp_1[gene],temp_1["prs_group"]))
            fe_or,fe_pv=fisher_exact(pd.crosstab(temp_1[gene],temp_1["prs_group"]))
            or_table_prs.append(fe_or)
            pvalue_table_prs.append(fe_pv)
            
            temp_1["tl_group"]=0
            temp_1.loc[(temp_1["tl_q"]==y),"tl_group"]=1
            fe_or,fe_pv=fisher_exact(pd.crosstab(temp_1[gene],temp_1["tl_group"]))
            or_table_tl.append(fe_or)
            pvalue_table_tl.append(fe_pv)
        
        prs_or_all[i].append(np.array(or_table_prs))
        prs_pv_all[i].append(np.array(pvalue_table_prs))
        tl_or_all[i].append(np.array(or_table_tl))
        tl_pv_all[i].append(np.array(pvalue_table_tl))   
    

In [ ]:
n=2
significance=[[] for i in range(n)]
significance_prs=[[] for i in range(n)]
significance_tl=[[] for i in range(n)]
for i in range(n):
    for j in range(4):
        print(i,j)
        s=np.full((2,2),"",dtype="str")
        s[pv_all[i][j]<0.05]="*"
        significance[i].append(s)
        
        s=np.full(2,"",dtype="str")
        s[prs_pv_all[i][j]<0.05]="*"
        significance_prs[i].append(s)

        s=np.full(2,"",dtype="str")
        s[tl_pv_all[i][j]<0.05]="*"
        significance_tl[i].append(s)
        
annot=[[] for i in range(n)]
annot_prs=[[] for i in range(n)]
annot_tl=[[] for i in range(n)]

for i in range(n):
    for j in range(4):
        annot[i].append(contigency_table_all[i][j].astype("str")+"\n"+significance[i][j])
        annot_prs[i].append(prs_counts[i][j].astype("str")+"\n"+significance_prs[i][j])
        annot_tl[i].append(tl_counts[i][j].astype("str")+"\n"+significance_tl[i][j])


In [ ]:


log_freq_all=[]
for x in freq_all:
    log_freq_all.append([np.log2(f) for f in x])

log_prs_freq_all=[[np.log2(x) for x in xs] for xs in prs_freq_all]
log_tl_freq_all=[[np.log2(x) for x in xs] for xs in tl_freq_all]

max_or=[x.max().max() for xs in or_all for x in xs]+[x.max() for xs in prs_or_all for x in xs]+[x.max() for xs in tl_or_all for x in xs]
min_or=[x.min().min() for xs in or_all for x in xs]+[x.min() for xs in prs_or_all for x in xs]+[x.min() for xs in tl_or_all for x in xs]

fig = plt.figure(figsize=(12,7))
gs0 = fig.add_gridspec(2,4, wspace=0.15, hspace=0.43)
ax=[[[] for i in range(4)] for j in range(2)]
for i in range(2):
    for j in range(4):
        spec=gs0[i,j].subgridspec(ncols=2, nrows=2,width_ratios=[1, 2], wspace=0.12, hspace=0.07, height_ratios=[ 2, 1])
        ax[i][j].append(fig.add_subplot(spec[0]))
        ax[i][j].append(fig.add_subplot(spec[1],sharey=ax[i][j][0]))
        ax[i][j].append(fig.add_subplot(spec[3],sharex=ax[i][j][1]))
        for k in [1,2]:
            ax[i][j][k].tick_params(labelleft=False)
        if j!=0:
            ax[i][j][0].tick_params(labelleft=False)
        for k in [0,1]:
            ax[i][j][k].tick_params(labelbottom=False)
        

#divnorm = colors.TwoSlopeNorm(vmin=min(min_or), vcenter=1,vmax= max(max_or))
divnorm = matplotlib.colors.TwoSlopeNorm(vmin=0.2, vcenter=1,vmax= 4.3)

current_cmap = matplotlib.colormaps['bwr'].copy()
cbar_ax = fig.add_axes([0.93, 0.15, 0.02, 0.7])

for i in range(2):
    for j,gene in enumerate(["DNMT3A_other","TET2","SRSF2","PPM1D"]):
        if i==1 and j==3:
            cbar=sns.heatmap(or_all[i][j][::-1], ax=ax[i][j][1], annot=significance[i][j][::-1], norm=divnorm, cmap=current_cmap, square=True,linewidths=0.18,linecolor = "gray",clip_on=False, cbar_ax=cbar_ax, fmt="s", annot_kws={'color':'k'},cbar_kws={'ticks':[0.25,0.5,0.75,1,2,3,4]})
        else:
            cbar=sns.heatmap(or_all[i][j][::-1], ax=ax[i][j][1], annot=significance[i][j][::-1], norm=divnorm, cmap=current_cmap, square=True,linewidths=0.18,linecolor = "gray",clip_on=False,cbar=False,fmt="s",annot_kws={'color':'k'})
        
        cbar=sns.heatmap(np.asarray(prs_or_all[i][j][::-1]).reshape(2,1), ax=ax[i][j][0], annot=np.asarray(significance_prs[i][j][::-1]).reshape(2,1), norm=divnorm, cmap=current_cmap, square=True,linewidths=0.18,linecolor = "gray",clip_on=False,cbar=False,fmt="s",annot_kws={'color':'k'})
        cbar=sns.heatmap(np.asarray(tl_or_all[i][j]).reshape(1,2), ax=ax[i][j][2], annot=np.asarray(significance_tl[i][j]).reshape(1,2), norm=divnorm, cmap=current_cmap, square=True,linewidths=0.18,linecolor = "gray",clip_on=False,cbar=False,fmt="s",annot_kws={'color':'k'})
        
        ax[i][j][1].set_ylabel("")
        ax[i][j][1].set_xlabel("")
        ax[i][j][2].set_xticks(np.arange(0.5,2.5))
        ax[i][j][2].set_xticklabels(np.arange(1,3))
        if j==0:
            ax[i][j][0].set_yticks(np.arange(0.5,2.5))
            ax[i][j][0].set_yticklabels(np.arange(1,3)[::-1])
        if i==0:
            ax[i][j][1].set_title(gene,pad=10)
        ax[i][j][0].set_xticks([])
        ax[i][j][2].set_yticks([])
fig.text(0.55, 0.04, 'Measured TL quantiles', ha='center',fontsize=15)
fig.text(0.08, 0.5, 'PRS quantiles', va='center', rotation='vertical',fontsize=15)
cbar_ax.tick_params(axis='y', which='both',direction='in',colors="k")
cbar_ax.set_yticklabels(['0.25', '0.5','0.75','1','2','3', '4'],fontsize=8,color="k")
cbar_ax.set_title('OR')
#plt.savefig("or_prs_tl_with_counts_2q_vaf_lt_0.1.pdf",bbox_inches="tight")
#plt.savefig("or_prs_tl_with_counts_2q_vaf_lt_0.1.svg",bbox_inches="tight")


fig.canvas.draw()
transFigure = fig.transFigure.inverted()

for i in [0,1]:
    coord1 = transFigure.transform(ax[0][i][1].transData.transform([2,0.5]))
    coord2 = transFigure.transform(ax[0][i][1].transData.transform([2.15,0.5]))
    coord3 = transFigure.transform(ax[1][i][1].transData.transform([2.15,0.5]))
    coord4 = transFigure.transform(ax[1][i][1].transData.transform([2,0.5]))

    line1 = matplotlib.lines.Line2D((coord1[0],coord2[0]),(coord1[1],coord2[1]), transform=fig.transFigure,color="k",linestyle="--",linewidth=0.7)
    line2 = matplotlib.lines.Line2D((coord2[0],coord3[0]),(coord2[1],coord3[1]), transform=fig.transFigure,color="k",linestyle="--",linewidth=0.7)
    line3 = matplotlib.lines.Line2D((coord3[0],coord4[0]),(coord3[1],coord4[1]), transform=fig.transFigure,color="k",linestyle="--",linewidth=0.5)

    fig.lines=fig.lines+[line1,line2,line3]
    
for i in [2,3]:
    coord1 = transFigure.transform(ax[0][i][1].transData.transform([2,1.5]))
    coord2 = transFigure.transform(ax[0][i][1].transData.transform([2.15,1.5]))
    coord3 = transFigure.transform(ax[1][i][1].transData.transform([2.15,1.5]))
    coord4 = transFigure.transform(ax[1][i][1].transData.transform([2,1.5]))

    line1 = matplotlib.lines.Line2D((coord1[0],coord2[0]),(coord1[1],coord2[1]), transform=fig.transFigure,color="k",linestyle="--",linewidth=0.7)
    line2 = matplotlib.lines.Line2D((coord2[0],coord3[0]),(coord2[1],coord3[1]), transform=fig.transFigure,color="k",linestyle="--",linewidth=0.7)
    line3 = matplotlib.lines.Line2D((coord3[0],coord4[0]),(coord3[1],coord4[1]), transform=fig.transFigure,color="k",linestyle="--",linewidth=0.5)

    fig.lines=fig.lines+[line1,line2,line3]
#plt.savefig("../figures_revision1/tl_prs_qq_2_singlegene_noHM.svg")
#plt.savefig("../figures_revision1/tl_prs_qq_2_singlegene_noHM.pdf")   
#plt.savefig("../figures_revision1/tl_prs_qq_2_singlegene_noHM.png")   

plt.show()

In [ ]:
########## Plot MR results############


mr_res=pd.read_csv("../polygenic_risk_score/farm/regression_coef/revision1_singlegene/tl_as_exp_ch_nowbc_flip_final_r1_singlegene.txt",usecols=["CH_type","method","or","or_lci95","or_uci95","pval"],header="infer",sep="\t")

display(mr_res)
mr_res=mr_res[(mr_res["method"]=="Inverse variance weighted") & (mr_res["CH_type"].isin(["dnmt3a","tet2","asxl1","ppm1d","splicing","mpn","long_telomere","short_telomere","all","Y_loss","X_loss","autosomal"]))]
display(mr_res)

mr_res=mr_res.sort_values('CH_type', key=lambda s: s.apply(["dnmt3a","tet2","asxl1","ppm1d","splicing","mpn","long_telomere","short_telomere","all","Y_loss","X_loss","autosomal"].index), ignore_index=True)
print(mr_res)
fig, axes = plt.subplots(figsize=(8,6.5))
axes.axvline(x=1,linestyle='--',linewidth=0.5)
data=mr_res.copy(deep=True).reset_index()

axes.errorbar(data.loc[data["pval"]<0.05,"or"],data[data["pval"]<0.05].index, xerr=[(data.loc[data["pval"]<0.05,"or"]-data.loc[data["pval"]<0.05,"or_lci95"]),(data.loc[data["pval"]<0.05,"or_uci95"]-data.loc[data["pval"]<0.05,"or"])], fmt='o',label="P value < 0.05",markersize=9,elinewidth=1,c="#483737")
display(data)
axes.errorbar(data.loc[(data["pval"]>=0.05),"or"],data[(data["pval"]>=0.05)].index, xerr=[(data.loc[(data["pval"]>=0.05),"or"]-data.loc[(data["pval"]>=0.05),"or_lci95"]),(data.loc[(data["pval"]>=0.05),"or_uci95"]-data.loc[(data["pval"]>=0.05),"or"])], fmt='o',label="P value "+r'$\mathrm{\geq} $'+" 0.05",markersize=9,elinewidth=1,c="#ac9393")
#axes.set_xlim([0,5])
axes.set_xlabel("OR",fontsize=15)
axes.set_ylim([-0.8,11.8])
axes.set_yticks(np.arange(12))
axes.set_yticklabels(["DNMT3A","TET2","ASXL1","PPM1D","Splicing","JAK2,CALR","Long telomere","Short telomere","All CH","LOY","LOX","autosomal"])
#axes.ticklabelparams(fontsize=16)
#axes.set_xticks([0.2,0.3,0.6,1,2,3,4,6])
#axes.set_xticklabels([0.2,0.3,0.6,1,2,3,4,6])
axes.legend(fontsize=12)


axes.set_xscale('log')
axes.tick_params(axis="both",which='major',labelsize=12)
axes.tick_params(axis="both",which='minor',labelsize=12)

#axes.get_xaxis().get_major_formatter().labelOnlyBase = False
axes.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
axes.get_xaxis().set_minor_formatter(matplotlib.ticker.ScalarFormatter())
axes.spines[['right', 'top']].set_visible(False)

#        k+=1
#plt.savefig("../figures_revision1/MR_flip_final_singlegene.png",bbox_inches="tight",dpi=600)
#plt.savefig("../figures_revision1/MR_flip_final_singlegene.pdf",bbox_inches="tight")
#plt.savefig("../figures_revision1/MR_flip_final_singlegene.svg",bbox_inches="tight")
plt.show()

In [ ]:
data=pd.concat([data,pd.DataFrame(multipletests(data["pval"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1])],axis=1)
data=data.rename(columns={0:"FDR_all"})

In [ ]:
data["CH_type"]=["DNMT3A","TET2","ASXL1","PPM1D","Splicing","JAK2,CALR","Long telomere","Short telomere","All CH","LOY","LOX","autosomal"]

In [ ]:
#data[[x for x in data.columns if (x!="index" and x!="method") ]].to_csv("mr_res_r1_singlegene.tsv",sep="\t",index=False)

In [ ]:
######################################Supplementary figures##################################

In [ ]:
######## Association with paternal age  ##################################

######### Load data ###################
parents_age=pd.read_csv("father_mother_age.tsv",sep="\t",dtype={'f.eid':"str"})
parents_age.columns=["eid","maternal_age","paternal_age"]
parents_age["maternal_age"]=parents_age["maternal_age"].map(lambda row:np.nan if row<0 else row)
parents_age["paternal_age"]=parents_age["paternal_age"].map(lambda row:np.nan if row<0 else row)
ukb500k_ts_som=ukb500k_ts_som.merge(parents_age,how="left",on="eid")
ukb500k_ts_som["paternal_age_at_birth"]=ukb500k_ts_som["paternal_age"]-ukb500k_ts_som["age"]
ukb500k_ts_som["maternal_age_at_birth"]=ukb500k_ts_som["maternal_age"]-ukb500k_ts_som["age"]
ukb500k_ts_som["paternal_age_at_birth"]=ukb500k_ts_som["paternal_age_at_birth"].map(lambda row:np.nan if row<5 else row)
ukb500k_ts_som["maternal_age_at_birth"]=ukb500k_ts_som["maternal_age_at_birth"].map(lambda row:np.nan if row<5 else row)

In [ ]:
#################### Define Long/short groups #################
ukb500k_ts_som["telomere_group"]="Control"
ukb500k_ts_som.loc[(ukb500k_ts_som["single_gene"].isin(["DNMT3A_other","DNMT3A_R882","JAK2","GNAS","GNB1","TET2","CBL"])),"telomere_group"]="Long"
ukb500k_ts_som.loc[(ukb500k_ts_som["single_gene"].isin(["PPM1D","SF3B1","SRSF2"])),"telomere_group"]="Short"
ukb500k_ts_som.loc[((ukb500k_ts_som["single_gene"]!="Control")  & (ukb500k_ts_som["telomere_group"]=="Control")),"telomere_group"]="Others"

In [ ]:
############### Perform association analysis ##############
temp=ukb500k_ts_som.copy(deep=True)

formula = 'telomere_group ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','paternal_age_at_birth'])
gene_list=list(temp["telomere_group"].unique())
gene_list.remove("Control")

or_table_paternal_age=[]

for gene in gene_list:
    data=temp[["paternal_age_at_birth", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","telomere_group"]].copy(deep=True)
    data=data.dropna()
    data=data[(data["telomere_group"].isin([gene,"Control"]))]
    data["telomere_group"]=data["telomere_group"].map({gene:1,"Control":0})
    print(data["telomere_group"].value_counts())
    lin_model=smf.logit(formula=formula,data=data)
    result=lin_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = np.exp(result.params)
    conf = np.exp(result.conf_int())
    print(result.pvalues)
    print([gene,params["paternal_age_at_birth"],conf.loc["paternal_age_at_birth",0],conf.loc["paternal_age_at_birth",1],result.pvalues["paternal_age_at_birth"]])
    or_table_paternal_age.append([gene,params["paternal_age_at_birth"],conf.loc["paternal_age_at_birth",0],conf.loc["paternal_age_at_birth",1],result.pvalues["paternal_age_at_birth"]])
or_table_paternal_age=pd.DataFrame(or_table_paternal_age)

In [ ]:
or_table_paternal_age

In [ ]:
########### Generate figure ######################3
plt.figure(figsize=(8,3))
or_table_paternal_age=or_table_paternal_age[or_table_paternal_age[0]!="Others"]
or_table_paternal_age[0]=["Long\n(2204)", "Short\n(98)"]

eb=plt.errorbar(or_table_paternal_age.loc[0,1],[0],xerr=[[or_table_paternal_age.loc[0,1]-or_table_paternal_age.loc[0,2]],[or_table_paternal_age.loc[0,3]-or_table_paternal_age.loc[0,1]]],fmt="o",label="P"+r'$\mathrm{<}$0.05', markersize=10,elinewidth=1,c="#73264d")
eb=plt.errorbar(or_table_paternal_age.loc[2,1],[1],xerr=[[or_table_paternal_age.loc[2,1]-or_table_paternal_age.loc[2,2]],[or_table_paternal_age.loc[2,3]-or_table_paternal_age.loc[2,1]]],fmt="o",label="P"+r'$\mathrm{\geq}0.05$', markersize=10,elinewidth=1,c="#df9fbf")


plt.yticks(np.arange(2),or_table_paternal_age[0])
plt.axvline(x=1,c="k",linestyle="--",linewidth=0.8)
plt.xlabel("OR",fontsize=16)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.ylim([-0.5,1.5])
plt.legend()

#plt.savefig("../figures_new/fathers_age_ass_ch_or.png",bbox_inches="tight",dpi=600)
#plt.savefig("../figures_revision1/fathers_age_assn_telomeregroup_or_singlegene.png",bbox_inches="tight",dpi=600)
#plt.savefig("../figures_revision1/fathers_age_assn_telomeregroup_or_singlegene.svg",bbox_inches="tight")
#plt.savefig("../figures_revision1/fathers_age_assn_telomeregroup_or_singlegene.pdf",bbox_inches="tight")

In [ ]:
or_table_paternal_age


In [ ]:
or_table_paternal_age.columns=["CH type","OR","OR lower CI (2.5%)","OR lower CI (97.5%)","P-value"]
or_table_paternal_age["CH type"]=["Long CH","Short CH"]
#or_table_paternal_age["FDR"]=multipletests(or_table_paternal_age["P-value"],alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
or_table_paternal_age.to_csv("SupplementaryTable_extfig2.ts",sep="\t",index=False)

In [ ]:
or_table_paternal_age

In [ ]:
######## Add time to diagnosis data for haematological malignancies ###################
tt_diagnosis=pd.read_csv("/home/sruthi/Downloads/tt_diagnosis.tsv",delimiter="\t",dtype={"eid":"str"})
tt_diagnosis.columns=[x+"_tt" for x in tt_diagnosis.columns]
tt_diagnosis=tt_diagnosis.rename(columns={"eid_tt":"eid"})
tt_diagnosis.replace(0,np.nan,inplace=True)
tt_diagnosis.columns

In [ ]:
############# Combine with main data set #####################
ukb500k_ts_som=ukb500k_ts_som.merge(tt_diagnosis,on="eid",how="left")

In [ ]:
#ukb500k_ts_som.drop(columns=['MPN', 'MDS', 'CMML', 'AML'],inplace=True)
ukb500k_ts_som["myeloid_malignancy"]=ukb500k_ts_som[["AML_tt","MDS_tt","MPN_tt","CMML_tt"]].apply(lambda row: "False" if row.isna().all() else row.idxmin(),axis=1)
ukb500k_ts_som["Time_to_malignancy"]=ukb500k_ts_som[['AML_tt','MDS_tt',"MPN_tt","CMML_tt"]].min(axis=1)
display(ukb500k_ts_som["myeloid_malignancy"].value_counts())
ukb500k_ts_som["myeloid_malignancy"]=ukb500k_ts_som["myeloid_malignancy"].map({"AML_tt":"AML","MDS_tt":"MDS","MPN_tt":"MPN","CMML_tt":"CMML","False":np.nan},na_action='ignore')
ukb500k_ts_som["Time_to_chemo"]=(ukb500k_ts_som["chemo_date"]-ukb500k_ts_som["daac"]).dt.days


In [ ]:
######## perform association analysis between myeloid malignancies and prs/ltl.
def run_reg(temp_data,MM_list_i,tl,formula):
    d=temp_data.copy(deep=True)
    d=d[(d["myeloid_malignancy"].isin(MM_list_i+["Control"]))]
    print(d["myeloid_malignancy"].value_counts(),MM_list_i)
    d["mm"]=d["myeloid_malignancy"].map(lambda row:1 if row in MM_list_i else 0)
    d=d.astype({"mm": int})
    print(formula)
    log_model=smf.glm(formula=formula,data=d,family=sm.families.Binomial())
    result=log_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = result.params
    conf = result.conf_int()
    return([np.exp(params[tl]),result.pvalues[tl],np.exp(conf.loc[tl,0]),np.exp(conf.loc[tl,1])])   
    

MM_list=[["MDS"],["AML"], ["MPN"],["MDS","MPN","CMML","AML"]]
names_MM=["MDS","AML","MPN","myeloid_malignancy"]

data_prs=ukb500k_ts_som.loc[(ukb500k_ts_som["Time_to_chemo"]>ukb500k_ts_som["Time_to_malignancy"]) |  (ukb500k_ts_som["Time_to_chemo"].isna()) |  (ukb500k_ts_som["Time_to_malignancy"].isna()),["total_PRS","sex", "age", "smoking_status","PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","myeloid_malignancy"]]

data_prs.loc[(data_prs["myeloid_malignancy"].isna()),"myeloid_malignancy"]="Control"
data_prs=data_prs.dropna()
print(data_prs["myeloid_malignancy"].value_counts())

data_tl=ukb500k_ts_som.loc[((ukb500k_ts_som["Time_to_chemo"]>ukb500k_ts_som["Time_to_malignancy"]) |  (ukb500k_ts_som["Time_to_chemo"].isna())  |  (ukb500k_ts_som["Time_to_malignancy"].isna())) & ((ukb500k_ts_som["HM_tt"]>0) |(ukb500k_ts_som["HM_tt"].isna()) ) & ((ukb500k_ts_som["Time_to_malignancy"].isna())|(ukb500k_ts_som["Time_to_malignancy"]>0)),["sex", "age", "smoking_status","PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","myeloid_malignancy","Z-adjusted T/S log"]]
data_tl=data_tl.rename(columns={"Z-adjusted T/S log":"tl"})


data_tl.loc[(data_tl["myeloid_malignancy"].isna()),"myeloid_malignancy"]="Control"
data_tl=data_tl.dropna()
print(data_tl["myeloid_malignancy"].value_counts())

#data=data.rename(columns={"Z-adjusted T/S log":"tl"})
formula_base = 'mm ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10'])

reg_coef_mm_all=[]
for i in range(4):
    
        reg_coef_mm_all.append(run_reg(data_prs,MM_list[i],"total_PRS",formula_base+"+total_PRS")+run_reg(data_tl,MM_list[i],"tl",formula_base+"+tl"))


reg_coef_mm_all=pd.DataFrame(reg_coef_mm_all)
reg_coef_mm_all.columns=["prs_or","prs_pvalue","prs_or_low_ci","prs_or_high_ci","tl_or","tl_pvalue","tl_or_low_ci","tl_or_high_ci"]
reg_coef_mm_all["MM"]=names_MM

display(reg_coef_mm_all)





plt.figure(figsize=(8,6))


reg_coef_mm_all_sel=reg_coef_mm_all.loc[(reg_coef_mm_all["prs_pvalue"]<0.05),:]
plt.errorbar(reg_coef_mm_all_sel.loc[:,"prs_or"],reg_coef_mm_all_sel.index,xerr=[reg_coef_mm_all_sel.loc[:,"prs_or"]-reg_coef_mm_all_sel.loc[:,"prs_or_low_ci"],reg_coef_mm_all_sel.loc[:,"prs_or_high_ci"]-reg_coef_mm_all_sel.loc[:,"prs_or"]],fmt="o",label="LTL-PRS (P"+r'$\mathrm{<}0.05$'+")",c="#7a7a52", markersize=9)


reg_coef_mm_all_sel=reg_coef_mm_all.loc[(reg_coef_mm_all["prs_pvalue"]>=0.05),:]
eb=plt.errorbar(reg_coef_mm_all_sel.loc[:,"prs_or"],reg_coef_mm_all_sel.index,xerr=[reg_coef_mm_all_sel.loc[:,"prs_or"]-reg_coef_mm_all_sel.loc[:,"prs_or_low_ci"],reg_coef_mm_all_sel.loc[:,"prs_or_high_ci"]-reg_coef_mm_all_sel.loc[:,"prs_or"]],fmt="o",label="LTL-PRS (P"+r'$\mathrm{\geq}0.05$'+")",c="#b8b894",markersize=9)
#eb[-1][0].set_linestyle('--')


reg_coef_mm_all_sel=reg_coef_mm_all.loc[(reg_coef_mm_all["tl_pvalue"]<0.05),:]
plt.errorbar(reg_coef_mm_all_sel.loc[:,"tl_or"],reg_coef_mm_all_sel.index+0.2,xerr=[reg_coef_mm_all_sel.loc[:,"tl_or"]-reg_coef_mm_all_sel.loc[:,"tl_or_low_ci"],reg_coef_mm_all_sel.loc[:,"tl_or_high_ci"]-reg_coef_mm_all_sel.loc[:,"tl_or"]],fmt="o",label="LTL (P"+r'$\mathrm{<}0.05$'+")", c="#cc6699",markersize=9)

#reg_coef_mm_all_sel=reg_coef_mm_all.loc[(reg_coef_mm_all["tl_pvalue"]>=0.05),:]
#eb=plt.errorbar(reg_coef_mm_all_sel.loc[:,"tl_or"],reg_coef_mm_all_sel.index+0.2,xerr=[reg_coef_mm_all_sel.loc[:,"tl_or"]-reg_coef_mm_all_sel.loc[:,"tl_or_low_ci"],reg_coef_mm_all_sel.loc[:,"tl_or_high_ci"]-reg_coef_mm_all_sel.loc[:,"tl_or"]],fmt="o",label="LTL-PRS (P"+r'$\mathrm{\geq}0.05$'+")",c="#0099cc",markersize=9)
#eb[-1][0].set_linestyle('--')

plt.yticks(np.arange(4)+0.1,names_MM,fontsize=13)
plt.xticks(fontsize=13)
plt.axvline(x=1,linestyle="--",linewidth=1)
plt.legend()

plt.xlabel("OR",fontsize=18)
#plt.savefig("../figures_final/MM_ch_all_tl_prs_OR_chemo.svg",dpi=600)
#plt.savefig("../figures_revision1/MM_ch_all_tl_prs_OR_chemo_noHM.svg",dpi=600)
#plt.savefig("../figures_revision1/MM_ch_all_tl_prs_OR_chemo_noHM.pdf",dpi=600)
#plt.savefig("../figures_revision1/MM_ch_all_tl_prs_OR_chemo_noHM.svg",dpi=600)



In [ ]:
prs_table=reg_coef_mm_all[["MM","prs_or","prs_pvalue","prs_or_low_ci","prs_or_high_ci"]].copy(deep=True)
prs_table["PRS/LTL"]="PRS"
tl_table=reg_coef_mm_all[["MM","tl_or","tl_pvalue","tl_or_low_ci","tl_or_high_ci"]].copy(deep=True)
tl_table.columns=["MM","prs_or","prs_pvalue","prs_or_low_ci","prs_or_high_ci"]
tl_table["PRS/LTL"]="LTL"

In [ ]:
supptable_ext_fig1=pd.concat([prs_table,tl_table])
supptable_ext_fig1.columns=["MM","OR","P-value","OR lower CI (2.5%)","OR upper CI (97.5%)","PRS/LTL"]
supptable_ext_fig1["FDR"]=multipletests(supptable_ext_fig1["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
supptable_ext_fig1[["PRS/LTL","MM","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value","FDR"]].to_csv("SupplementaryTable_extfig1.tsv",sep="\t",index=False)

In [ ]:
supptable_ext_fig1[["PRS/LTL","MM","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value","FDR"]]

In [ ]:
############################ Validation ##########################

In [ ]:
################# Topmed-derived PRS #################
temp=ukb500k_ts_som_mca[["topmed_all_trans_total_PRS","sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","CHR_COPY_CHANGE"]].copy(deep=True)
print(temp.shape)
temp=temp[~(temp["CHR_COPY_CHANGE"].isin(["Y:neutral","X:neutral","Y:gain","X:gain"]))]
temp=temp.dropna()
temp["MCA_type"]=temp["CHR_COPY_CHANGE"].map(lambda row:"autosomal MCA" if row not in ["Y:loss","X:loss","Control"] else row)
y="topmed_all_trans_total_PRS"

counts=temp["MCA_type"].value_counts()
print(counts.sum())
gene_order=["autosomal MCA","Y:loss","X:loss"]
plt.figure(figsize=(6,4))
b=sns.violinplot(data=temp,x="MCA_type",y=y,order=["Control"]+gene_order,linewidth=2,palette={"Control":gene_color["Control"],"autosomal MCA":"#999999","Y:loss":"#E69F00","X:loss":"#56B4E9"})
ax=plt.gca()
xticks=ax.get_xticklabels()
xticks_new=[]
for i in xticks:
    print(i.get_text(),counts[i.get_text()])
    xticks_new.append(i.get_text()+"\n("+str(counts[i.get_text()])+")")
#plt.ylim([-5,5])
plt.xticks(np.arange(0,len(xticks),1),xticks_new)
plt.axhline(y=temp.loc[temp["MCA_type"]=="Control",y].median(), color='k', linestyle='--')
plt.grid(axis='x')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.xaxis.label.set_size(18)
ax.yaxis.label.set_size(16)
plt.ylabel("PRS")
plt.xlabel("")
plt.show()

plot_values=[]
for mca_type in ["autosomal MCA","Y:loss","X:loss"]:
    if mca_type=="autosomal MCA":
        data=temp[temp["MCA_type"].isin(["Control",mca_type])].copy(deep=True)
        print(data.shape)
        formula = 'MCA_type ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"topmed_all_trans_total_PRS"])
    elif mca_type=="Y:loss":
        data=temp[(temp["MCA_type"].isin(["Control",mca_type])) & (temp["sex"]=="Male")].copy(deep=True)
        formula = 'MCA_type ~' + ' + '.join(['age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"topmed_all_trans_total_PRS"])
    else:
        data=temp[(temp["MCA_type"].isin(["Control",mca_type])) & (temp["sex"]=="Female")].copy(deep=True)
        formula = 'MCA_type ~' + ' + '.join(['age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"topmed_all_trans_total_PRS"])
    data["MCA_type"]=data["MCA_type"].map({mca_type:1,"Control":0})
    data["MCA_type"]=data["MCA_type"].astype(int)
    
    lin_model=smf.logit(formula=formula,data=data)
    result=lin_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = np.exp(result.params)
    conf = np.exp(result.conf_int())
    print(result.pvalues)
    plot_values.append([params["topmed_all_trans_total_PRS"],conf.loc["topmed_all_trans_total_PRS",0],conf.loc["topmed_all_trans_total_PRS",1],result.pvalues["topmed_all_trans_total_PRS"]])

plot_values=np.array(plot_values)
data_mca=temp.copy(deep=True)
data_mca=data_mca.rename(columns={"Z-adjusted T/S log":"tl"})
            

In [ ]:
supptable_topmed_mca=pd.DataFrame(plot_values)
supptable_topmed_mca["Gene/mCA"]=["autosomal mCA","LOY","LOX"]
supptable_topmed_mca.columns=["OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value","Gene/mCA"]
supptable_topmed_mca=supptable_topmed_mca[["Gene/mCA","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value"]]
supptable_topmed_mca["FDR"]=multipletests(supptable_topmed_mca["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

In [ ]:
############# Association with all CH and PRS ###############
y="topmed_all_trans_total_PRS"
temp=ukb500k_ts_som.copy(deep=True)
temp=temp.loc[(temp[y].notna())]

formula = 'single_gene ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','topmed_all_trans_total_PRS'])

or_table_pr=[]

data=temp[["topmed_all_trans_total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10", "single_gene"]].copy(deep=True)
data=data.dropna()
print(data.shape)
data["single_gene"]=data["single_gene"].map(lambda row:0 if row=="Control" else 1)
data["single_gene"]=data["single_gene"].astype(int)
print(data["single_gene"].value_counts())
log_model_allch=smf.logit(formula=formula,data=data)
result_allch=log_model_allch.fit(max_iterations= 100000)
display(result_allch.summary2())
params_allch = np.exp(result_allch.params)
conf_allch = np.exp(result_allch.conf_int())
print(result_allch.pvalues)
print(["all CH", params_allch["topmed_all_trans_total_PRS"], conf_allch.loc["topmed_all_trans_total_PRS",0], conf_allch.loc["topmed_all_trans_total_PRS",1], result_allch.pvalues["topmed_all_trans_total_PRS"]])
or_table_pr.append(["all CH", params_allch["topmed_all_trans_total_PRS"], conf_allch.loc["topmed_all_trans_total_PRS",0], conf_allch.loc["topmed_all_trans_total_PRS",1], result_allch.pvalues["topmed_all_trans_total_PRS"]])


In [ ]:
y="topmed_all_trans_total_PRS"
temp=ukb500k_ts_som.copy(deep=True)
temp=temp.loc[(temp[y].notna())]

formula = 'single_gene ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','topmed_all_trans_total_PRS'])
gene_list=list(temp["single_gene"].unique())
gene_list.remove("Control")

for gene in gene_list:
    data=temp[["topmed_all_trans_total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","single_gene"]].copy(deep=True)
    data=data.dropna()
    print(data.shape)
    data=data[(data["single_gene"].isin([gene,"Control"]))]
    data["single_gene"]=data["single_gene"].map({gene:1,"Control":0})
    print(data["single_gene"].value_counts())
    log_model=smf.logit(formula=formula,data=data)
    result=log_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = np.exp(result.params)
    conf = np.exp(result.conf_int())
    print(result.pvalues)
    print([gene,params["topmed_all_trans_total_PRS"],conf.loc["topmed_all_trans_total_PRS",0],conf.loc["topmed_all_trans_total_PRS",1],result.pvalues["topmed_all_trans_total_PRS"]])
    or_table_pr.append([gene,params["topmed_all_trans_total_PRS"],conf.loc["topmed_all_trans_total_PRS",0],conf.loc["topmed_all_trans_total_PRS",1],result.pvalues["topmed_all_trans_total_PRS"]])
or_table_pr=pd.DataFrame(or_table_pr)

In [ ]:
or_table_pr

In [ ]:
plt.errorbar(np.arange(15),or_table_pr.loc[:,1],yerr=[or_table_pr.loc[:,1]-or_table_pr.loc[:,2],or_table_pr.loc[:,3]-or_table_pr.loc[:,1]],fmt="o",markersize=7)
ticklabel=or_table_pr[0]
ticklabel=[ticklabel[i]+"\n*" if or_table_pr.loc[i,4]<0.05 else ticklabel[i] for i in range(15)]
plt.xticks(np.arange(15),ticklabel,fontsize=13,rotation=90)
plt.ylabel("OR",fontsize=15)
plt.axhline(y=1,linestyle="--",lw=1)
#plt.savefig("topmed_all_trans_total_prs_or.png",dpi=600,bbox_inches="tight")

In [ ]:
y="topmed_all_trans_total_PRS"
temp=ukb500k_ts_som[["topmed_all_trans_total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","single_gene"]].copy(deep=True)
temp=temp.dropna()
gene_count=Counter(temp["single_gene"])

left, width = 0.01, 15*0.05
bottom_h, height_h=0.1,0.23
bottom, height = 0.35, .60
left_h = left+width+0.03
rect_0 = [left, bottom, width, height]
rect_1 = [left_h, bottom, 4*0.05, height]
rect_2 = [left, bottom_h, width, height_h]
rect_3 = [left_h, bottom_h, 4*0.05, height_h]

plt.figure(figsize=(22,12))
ax1=plt.axes(rect_0)
ax2=plt.axes(rect_1)
ax3=plt.axes(rect_2,sharex=ax1)
ax = [ax1, ax2, ax3, plt.axes(rect_3,sharex=ax2)]

gene_color_new={}
for key in gene_color:
    gene_color_new[key]=sns.set_hls_values(gene_color[key],s=0.5)
black=sns.set_hls_values('#000000',s=0.5)

gene_list=[]
gene_groups=[]
quartiles=[]

gene_list.append("Control")
gene_groups.append(np.array(temp.loc[((temp["single_gene"]=="Control") & (temp[y].notna())),y]))
quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))

gene_list.append("all CH")
gene_groups.append(np.array(temp.loc[((temp["single_gene"].isin(["ASXL1","CALR","CBL","DNMT3A_R882","DNMT3A_other","GNAS", "GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1"])) & (temp[y].notna())),y]))
quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))

for key in ["ASXL1","CALR","CBL","DNMT3A_R882","DNMT3A_other","GNAS", "GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1"]: 
        if gene_count[key]>=20:
            gene_list.append(key)
            gene_groups.append(np.array(temp.loc[((temp["single_gene"]==key) & (temp[y].notna())),y]))
            quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))
quartiles=np.array(quartiles)           
            
if gene_list != []:
        temp=temp.loc[((temp["single_gene"].isin(gene_list)) & (temp[y].notna())),:]
        counts=temp["single_gene"].value_counts()
        counts["all CH"]=gene_groups[1].shape[0]
        
        parts = ax[0].violinplot(gene_groups, showmeans=False, showmedians=False, showextrema=False,widths=0.65,positions=np.arange(0,len(gene_list),1))

        for i,pc in enumerate(parts['bodies']):
            if gene_list[i]=="Control":
                pc.set_facecolor(gene_color_new["Control"])
            else:
                pc.set_facecolor("#999999")
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)

        ax[0].boxplot(gene_groups,positions=np.arange(0,len(gene_list),1),showcaps=False,widths=0.06, patch_artist=True, boxprops=dict(color=black, facecolor=black),whiskerprops=dict(color=black, linewidth=2), medianprops=dict(color="w", linewidth=0 ),showfliers=False)
        inds = np.arange(0, len(quartiles))
        ax[0].scatter(inds, quartiles[:,1], marker='o', color='white', s=30, zorder=3)
        
        xticks_new=[]
        for i in gene_list:
            print(i,counts[i])
            if i != "Control":
                pval_prs=or_table_pr.loc[(or_table_pr[0]==i),4].reset_index()
                pval_prs=pval_prs.iloc[0,1]
            else:
                pval_prs=1
            if pval_prs <0.05:
                xticks_new.append(i[0].upper()+i[1:]+"\n("+str(counts[i])+")\n*")
            else:
                xticks_new.append(i[0].upper()+i[1:]+"\n("+str(counts[i])+")")
        print(xticks_new)
        ax[0].set_xticks(np.arange(0,len(gene_list),1))
        #ax[0].set_xticklabels(xticks_new,rotation=45)
        ax[0].set_xticklabels([])
        ax[0].tick_params(axis='y',labelsize=15,size=7)
        ax[0].tick_params(axis='x',labelsize=12,labelbottom=False)
        ax[0].grid(axis='x')
        ax[0].xaxis.label.set_size(20)
        ax[0].yaxis.label.set_size(22)
        ax[0].set_ylabel("PRS")
        ax[0].set_ylim([-4,4])
        #ax[0].set_xlabel("Gene")
        ax[0].axhline(y=temp.loc[temp["single_gene"]=="Control",y].median(), color='k', linestyle='--')
        print(temp.loc[temp["single_gene"]=="Control",y].median())
                        
temp=data_mca[[y,"MCA_type"]].copy(deep=True)
temp=temp.dropna()
temp["MCA_type"]=temp["MCA_type"].map(lambda row:"autosomal" if row not in ["Y:loss","X:loss","Control"] else row)
y="topmed_all_trans_total_PRS"

temp=temp.loc[(temp[y].notna()),:]
counts=temp["MCA_type"].value_counts()
print(counts.sum())
gene_groups=[]
gene_order=["Control","autosomal","Y:loss","X:loss"]
quartiles=[]
for key in gene_order:
    gene_groups.append(temp.loc[(temp["MCA_type"]==key)& (temp[y].notna()),y])
    quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))
quartiles=np.array(quartiles)           
                       
parts = ax[1].violinplot(gene_groups, showmeans=False, showmedians=False, showextrema=False,widths=0.65,positions=np.arange(0,len(gene_order),1))
mca_color={"Control":gene_color["Control"],"autosomal":"#999999","Y:loss":"#E69F00","X:loss":"#56B4E9"}           
for key in mca_color:
    mca_color[key]=sns.set_hls_values(mca_color[key],s=0.5)
    
for i,pc in enumerate(parts['bodies']):
            if gene_order[i]=="Control":
                pc.set_facecolor(mca_color[gene_order[i]])
            else:
                pc.set_facecolor("#8585ad")
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)

ax[1].boxplot(gene_groups,positions=np.arange(0,len(gene_order),1),showcaps=False,widths=0.06, patch_artist=True,boxprops=dict(color=black, facecolor=black),
            whiskerprops=dict(color=black, linewidth=2),
            medianprops=dict(color="w", linewidth=0 ),showfliers=False)                       
inds = np.arange(0, len(quartiles))
ax[1].scatter(inds, quartiles[:,1], marker='o', color='white', s=30, zorder=3)                       
xticks_new_mca=[]
gene_rename={"autosomal":"autosomal","Y:loss":"LOY","X:loss":"LOX"}
for j,i in enumerate(gene_order):
    print(i,counts[i])
    if i!="Control":
        if plot_values[j-1,3]<0.05:
            xticks_new_mca.append(gene_rename[i]+"\n("+str(counts[i])+")"+"\n*")
        else:
            xticks_new_mca.append(gene_rename[i]+"\n("+str(counts[i])+")")
    else:
        xticks_new_mca.append(i+"\n("+str(counts[i])+")")

ax[1].set_xticks(np.arange(0,len(xticks),1))
ax[1].set_xticklabels([])
ax[1].grid(axis='x')
ax[1].tick_params(axis='x',labelsize=12,labelbottom=False)
ax[1].tick_params(axis='y',labelsize=15)
ax[1].xaxis.label.set_size(25)
ax[1].yaxis.label.set_size(25)
ax[1].set_ylim([-4,4])

ax[1].axhline(y=temp.loc[temp["MCA_type"]=="Control",y].median(), color='k', linestyle='--')
plt.subplots_adjust(wspace=0.05)

#or_table_pr=or_table_pr.set_index(0).loc[[x for x in gene_list if x!="Control"],:]
#or_table_pr=or_table_pr.reset_index()
or_table_pr=or_table_pr.set_index(0).loc[[x for x in gene_list if x!="Control"],:]
or_table_pr=or_table_pr.reset_index()
or_significant=or_table_pr.loc[(or_table_pr[4]>=0.05),:]
ax[2].errorbar(or_significant.index+1,or_significant.loc[:,1],yerr=[or_significant.loc[:,1]-or_significant.loc[:,2],or_significant.loc[:,3]-or_significant.loc[:,1]],fmt="o",markersize=20,elinewidth=2,c="#b3b3b3",label="P"+r'$\mathrm{\geq}0.05$')
or_significant=or_table_pr.loc[(or_table_pr[4]<0.05),:]
ax[2].errorbar(or_significant.index+1,or_significant.loc[:,1],yerr=[or_significant.loc[:,1]-or_significant.loc[:,2],or_significant.loc[:,3]-or_significant.loc[:,1]],fmt="o",markersize=20,elinewidth=2,c="#4d4d4d",label="P"+r'$\mathrm{<}0.05$')

ax[2].set_xticks(np.arange(0,16))
ax[2].set_xticklabels(xticks_new,fontsize=15,rotation=35)
ax[2].set_ylabel("OR",fontsize=22)
ax[2].set_xlabel("CH driver gene",fontsize=22)
ax[2].tick_params(axis='y',labelsize=15,size=7)
ax[2].axhline(y=1,linestyle="--",lw=1)
ax[2].legend()


plot_values_s=plot_values[(plot_values[:,3]<0.05),:]
ax[3].errorbar(np.where(plot_values[:,3]<0.05)[0]+1,plot_values_s[:,0],yerr=[plot_values_s[:,0]-plot_values_s[:,1],plot_values_s[:,2]-plot_values_s[:,0]],fmt="o",markersize=20,elinewidth=2,c="#575782",label="P"+r'$\mathrm{<}0.05$')

plot_values_n=plot_values[plot_values[:,3]>=0.05,:]
ax[3].errorbar(np.where(plot_values[:,3]>=0.05)[0]+1,plot_values_n[:,0],yerr=[plot_values_n[:,0]-plot_values_n[:,1],plot_values_n[:,2]-plot_values_n[:,0]],fmt="o",markersize=20,elinewidth=2,c="#b1b1cb",label="P"+r'$\mathrm{\geq}0.05$')
ax[3].set_xticks(np.arange(0,4))
ax[3].set_xticklabels(xticks_new_mca,fontsize=15,rotation=45)
ax[3].tick_params(axis='y',labelsize=15)
ax[3].axhline(y=1,linestyle="--",lw=1)
ax[3].set_xlabel("mCA type",fontsize=22)
ax[3].legend()

for a in ax:
    a.spines[['right', 'top']].set_visible(False)

#plt.savefig("../figures_revision1/topmedprs_ch_or_mca_singlegene.png",dpi=600,bbox_inches="tight")
#plt.savefig("../figures_revision1/topmedprs_ch_or_mca_singlegene.pdf",bbox_inches="tight")
#plt.savefig("../figures_revision1/topmedprs_ch_or_mca_singlegene.svg",bbox_inches="tight")

plt.show()
            


In [ ]:
supptable_topmed_gene=or_table_pr.copy(deep=True)
supptable_topmed_gene.columns=["Gene/mCA","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value"]
supptable_topmed_gene=supptable_topmed_gene.sort_values(by="Gene/mCA",key=lambda col: col.str.lower())
supptable_topmed_gene["FDR"]=multipletests(supptable_topmed_gene["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
supptable_topmed=pd.concat([supptable_topmed_gene[["Gene/mCA","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value","FDR"]],supptable_topmed_mca[["Gene/mCA","OR","OR lower CI (2.5%)","OR upper CI (97.5%)","P-value","FDR"]]])
supptable_topmed["FDR_all"]=multipletests(supptable_topmed["P-value"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]


In [ ]:
supptable_topmed.to_csv("prs_coef_topmed_supptable_r1_singlegene_check.tsv",sep="\t",index=False)

In [ ]:
supptable_topmed

In [ ]:
######### Load AllofUs data ############
aou_highvaf_prs=pd.read_csv("../allofus/ltl_prs_zscores_EUR_aou_09162024.tsv",delimiter="\t")
aou_highvaf_or=pd.read_csv("../allofus/ltl_prs_OR_by_gene_EUR_aou_09162024.tsv",delimiter="\t")
aou_singlemut_prs=pd.read_csv("../allofus/ltl_prs_zscores_EUR_aou_singlemutant_09202024.tsv",delimiter="\t")
aou_singlemut_or=pd.read_csv("../allofus/ltl_prs_OR_by_gene_EUR_aou_singlemutant_09202024.tsv",delimiter=",")
aou_singlemut_or=aou_singlemut_or[["Gene","OR","CI_lower","CI_upper","P","N"]]

aou_highvaf_or.loc[len(aou_highvaf_or.index)] = ['Splicing', 0.91665, 0.832261,1.009595,0.077368,391]
aou_highvaf_or.loc[len(aou_highvaf_or.index)] = ['Splicing+PPM1D', 0.916923, 0.832506,1.009901,0.078403,754]
aou_singlemut_or.loc[len(aou_singlemut_or.index)] = ['Splicing',0.919832,0.823256,1.027737,0.139799,296]
aou_singlemut_or.loc[len(aou_singlemut_or.index)] = ['Splicing+PPM1D',0.92009,0.82349,1.028021,0.141118,603]

In [ ]:
aou_highvaf_or[aou_highvaf_or["Gene"].isin(["Control","ASXL1","CBL","DNMT3A","GNAS", "GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1","Splicing","Splicing+PPM1D"])]

In [ ]:
supptable_aou_gene=aou_highvaf_or[aou_highvaf_or["Gene"].isin(["Control","ASXL1","CBL","DNMT3A","GNAS", "GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1","Splicing","Splicing+PPM1D"])].copy(deep=True)
supptable_aou_gene["FDR"]=multipletests(supptable_aou_gene["P"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

#supptable_aou_gene.to_csv("supptable_aou_highvaf_r1.tsv",sep="\t",index=False)

In [ ]:
supptable_aou_gene

In [ ]:
supptable_aou_gene=aou_singlemut_or[aou_singlemut_or["Gene"].isin(["Control","ASXL1","CBL","DNMT3A","GNAS", "GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1","Splicing","Splicing+PPM1D"])].copy(deep=True)
supptable_aou_gene["FDR"]=multipletests(supptable_aou_gene["P"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
supptable_aou_gene.loc[supptable_aou_gene["Gene"].isin(["ASXL1","CBL","GNAS", "GNB1","PPM1D","SF3B1","SRSF2","TP53","U2AF1"]),"FDR_selected"]=multipletests(supptable_aou_gene.loc[supptable_aou_gene["Gene"].isin(["ASXL1","CBL","GNAS", "GNB1","PPM1D","SF3B1","SRSF2","TP53","U2AF1"]),"P"], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]
#supptable_aou_gene.to_csv("supptable_aou_singlemut_r1.tsv",sep="\t",index=False)

In [ ]:
y="ltl_prs_zscore"
temp=aou_singlemut_prs.copy(deep=True)
temp=temp.rename(columns={"Gene.refGene":"single_gene"})
temp=temp.dropna()
gene_count=Counter(temp["single_gene"])


fig, ax = plt.subplots(2, 1,sharex=False,sharey=False,figsize=(23,12),gridspec_kw={'height_ratios':[5,1.5]})


gene_color_new={}
for key in gene_color:
    gene_color_new[key]=sns.set_hls_values(gene_color[key],s=0.5)
black=sns.set_hls_values('#000000',s=0.5)

gene_list=[]
gene_groups=[]
quartiles=[]
for key in ["Control","ASXL1","CBL","DNMT3A","GNAS", "GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1","Splicing","Splicing+PPM1D"]: 
            gene_list.append(key)
            if key == "Splicing":
                gene_groups.append(np.array(temp.loc[((temp["single_gene"].isin(["SF3B1","SRSF2","U2AF1"])) & (temp[y].notna())),y]))
            elif key=="Splicing+PPM1D":
                gene_groups.append(np.array(temp.loc[((temp["single_gene"].isin(["SF3B1","SRSF2","U2AF1","PPM1D"])) & (temp[y].notna())),y]))
            else:
                gene_groups.append(np.array(temp.loc[((temp["single_gene"]==key) & (temp[y].notna())),y]))
            quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))
quartiles=np.array(quartiles)
print(gene_list)

or_table_pr=aou_singlemut_or.set_index("Gene").loc[[x for x in gene_list if x!="Control"],:]
or_table_pr=or_table_pr.reset_index()

if gene_list != []:
        temp=temp.loc[((temp["single_gene"].isin(gene_list)) & (temp[y].notna())),:]
        counts=temp["single_gene"].value_counts()
        counts["Splicing"]=gene_groups[-2].shape[0]
        counts["Splicing+PPM1D"]=gene_groups[-1].shape[0]
        print(counts)
        
        parts = ax[0].violinplot(gene_groups, showmeans=False, showmedians=False, showextrema=False,widths=0.65,positions=np.arange(0,len(gene_list),1))

        for i,pc in enumerate(parts['bodies']):
            if gene_list[i]=="Control":
                pc.set_facecolor(gene_color_new["Control"])
            else:
                pc.set_facecolor("#999999")
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)

        ax[0].boxplot(gene_groups,positions=np.arange(0,len(gene_list),1),showcaps=False,widths=0.06, patch_artist=True, boxprops=dict(color=black, facecolor=black),whiskerprops=dict(color=black, linewidth=2), medianprops=dict(color="w", linewidth=0 ),showfliers=False)
        inds = np.arange(0, len(quartiles))
        ax[0].scatter(inds, quartiles[:,1], marker='o', color='white', s=30, zorder=3)
        
        xticks_new=[]
        for i in gene_list:
            print(i,counts[i])
            if i != "Control":
                pval_prs=or_table_pr.loc[(or_table_pr["Gene"]==i),"P"].reset_index()
                pval_prs=pval_prs.iloc[0,1]
            else:
                pval_prs=1
            if pval_prs <0.05:
                xticks_new.append(i+"\n("+str(counts[i])+")")
            else:
                xticks_new.append(i+"\n("+str(counts[i])+")")
        print(xticks_new)
        ax[0].set_xticks(np.arange(0,len(gene_list),1))
        #ax[0].set_xticklabels(xticks_new,rotation=45)
        ax[0].set_xticklabels([])
        ax[0].tick_params(axis='y',labelsize=15,size=7)
        ax[0].tick_params(axis='x',labelsize=12,labelbottom=False)
        ax[0].grid(axis='x')
        ax[0].xaxis.label.set_size(20)
        ax[0].yaxis.label.set_size(22)
        ax[0].set_ylabel("PRS")
        ax[0].set_ylim([-4,4])
        #ax[0].set_xlabel("Gene")
        ax[0].axhline(y=temp.loc[temp["single_gene"]=="Control",y].median(), color='k', linestyle='--')
        print(temp.loc[temp["single_gene"]=="Control",y].median())
                        


or_significant=or_table_pr.loc[(or_table_pr["P"]>=0.05),:]
print(or_significant)
ax[1].errorbar(or_significant.index+1,or_significant.loc[:,"OR"],yerr=[or_significant.loc[:,"OR"]-or_significant.loc[:,"CI_lower"],or_significant.loc[:,"CI_upper"]-or_significant.loc[:,"OR"]],fmt="o",markersize=15,elinewidth=4,c="#b3b3b3",label="P"+r'$\mathrm{\geq}0.05$')
or_significant=or_table_pr.loc[(or_table_pr["P"]<0.05),:]
ax[1].errorbar(or_significant.index+1,or_significant.loc[:,"OR"],yerr=[or_significant.loc[:,"OR"]-or_significant.loc[:,"CI_lower"],or_significant.loc[:,"CI_upper"]-or_significant.loc[:,"OR"]],fmt="o",markersize=15,elinewidth=4,c="#4d4d4d",label="P"+r'$\mathrm{<}0.05$')


ax[1].set_xticks(np.arange(0,15))
ax[1].set_xticklabels(xticks_new,fontsize=15,rotation=35)
ax[1].set_ylabel("OR",fontsize=22)
ax[1].set_xlabel("CH driver gene",fontsize=22)
ax[1].tick_params(axis='y',labelsize=15,size=7)
ax[1].axhline(y=1,linestyle="--",lw=1)
ax[1].legend()


for a in ax:
    a.spines[['right', 'top']].set_visible(False)

#plt.savefig("../figures_revision1/prs_ch_singlemut_or.png",dpi=600,bbox_inches="tight")
#plt.savefig("../figures_revision1/prs_ch_singlemut_or.pdf",bbox_inches="tight")
#plt.savefig("../figures_revision1/prs_ch_singlemut_or.svg",bbox_inches="tight")


plt.show()
            


In [ ]:
############################ Response to reviewer ###########################

In [ ]:

############# Association with all CH and PRS for all genes with n>50 ###############
y="total_PRS"
temp=ukb500k_ts_som_allgenes.copy(deep=True)
temp=temp.loc[(temp[y].notna())]

formula = 'single_gene ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','total_PRS'])

or_table_pr=[]

data=temp[["total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10", "single_gene"]].copy(deep=True)
data=data.dropna()
print(data.shape)
data["single_gene"]=data["single_gene"].map(lambda row:0 if row=="Control" else 1)
data["single_gene"]=data["single_gene"].astype(int)
print(data["single_gene"].value_counts())
log_model_allch=smf.logit(formula=formula,data=data)
result_allch=log_model_allch.fit(max_iterations= 100000)
display(result_allch.summary2())
params_allch = np.exp(result_allch.params)
conf_allch = np.exp(result_allch.conf_int())
print(result_allch.pvalues)
print(["all CH", params_allch["total_PRS"], conf_allch.loc["total_PRS",0], conf_allch.loc["total_PRS",1], result_allch.pvalues["total_PRS"]])
or_table_pr.append(["all CH", params_allch["total_PRS"], conf_allch.loc["total_PRS",0], conf_allch.loc["total_PRS",1], result_allch.pvalues["total_PRS"]])

############# Association with CH and PRS for all genes with n>50 ###############


y="total_PRS"
temp=ukb500k_ts_som_allgenes.copy(deep=True)
temp=temp.loc[(temp[y].notna())]

formula = 'single_gene ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','total_PRS'])
gene_list=list(temp["single_gene"].unique())
gene_list.remove("Control")

for gene in gene_list:
    data=temp[["total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","single_gene"]].copy(deep=True)
    data=data.dropna()
    print(data.shape)
    data=data[(data["single_gene"].isin([gene,"Control"]))]
    data["single_gene"]=data["single_gene"].map({gene:1,"Control":0})
    data["single_gene"]=data["single_gene"].astype(int)
    print(data["single_gene"].value_counts())
    log_model=smf.logit(formula=formula,data=data)
    result=log_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = np.exp(result.params)
    conf = np.exp(result.conf_int())
    print(result.pvalues)
    print([gene,params["total_PRS"],conf.loc["total_PRS",0],conf.loc["total_PRS",1],result.pvalues["total_PRS"]])
    or_table_pr.append([gene,params["total_PRS"],conf.loc["total_PRS",0],conf.loc["total_PRS",1],result.pvalues["total_PRS"]])
or_table_pr=pd.DataFrame(or_table_pr)

or_table_pr

or_table_pr

In [ ]:
y="total_PRS"
temp=ukb500k_ts_som_allgenes[["total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","single_gene"]].copy(deep=True)
temp=temp.dropna()
gene_count=Counter(temp["single_gene"])

"""
left, width = 0.02, 15*0.05
bottom_h, height_h=0.1,0.23
bottom, height = 0.35, .60
left_h = left+width+0.02
rect_0 = [left, bottom, width, height]
rect_1 = [left_h, bottom, 4*0.05, height]
rect_2 = [left, bottom_h, width, height_h]
rect_3 = [left_h, bottom_h, 4*0.05, height_h]

plt.figure(figsize=(22,12))
ax1=plt.axes(rect_0)
ax2=plt.axes(rect_1)
ax3=plt.axes(rect_2,sharex=ax1)
ax = [ax1, ax2, ax3, plt.axes(rect_3,sharex=ax2)]
"""

fig, ax = plt.subplots(2, 1,sharex=False,sharey=False,figsize=(26,12),gridspec_kw={'height_ratios':[5,1.5]})

gene_color_new={}
for key in gene_color_allgenes:
    gene_color_new[key]=sns.set_hls_values(gene_color_allgenes[key],s=0.5)
black=sns.set_hls_values('#000000',s=0.5)

gene_list=[]
gene_groups=[]
quartiles=[]

gene_list.append("Control")
gene_groups.append(np.array(temp.loc[((temp["single_gene"]=="Control") & (temp[y].notna())),y]))
quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))

gene_list.append("all CH")
gene_groups.append(np.array(temp.loc[((temp["single_gene"].isin(["ASXL1","CALR","CBL","DNMT3A_other","DNMT3A_R882","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1","ATM","STAT3","BRCC3","CHEK2","KDM6A","KRAS","BCOR", "STAG2","BCORL1","IDH2"])) & (temp[y].notna())),y]))
quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))

for key in ["ASXL1","CALR","CBL","DNMT3A_other","DNMT3A_R882","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1","ATM","STAT3","BRCC3","CHEK2","KDM6A","KRAS","BCOR", "STAG2","BCORL1","IDH2"]: 
        if gene_count[key]>=20:
            gene_list.append(key)
            gene_groups.append(np.array(temp.loc[((temp["single_gene"]==key) & (temp[y].notna())),y]))
            quartiles.append(np.percentile(gene_groups[-1],[25, 50, 75]))
quartiles=np.array(quartiles)           
            
if gene_list != []:
        temp=temp.loc[((temp["single_gene"].isin(gene_list)) & (temp[y].notna())),:]
        counts=temp["single_gene"].value_counts()
        counts["all CH"]=gene_groups[1].shape[0]
        
        parts = ax[0].violinplot(gene_groups, showmeans=False, showmedians=False, showextrema=False,widths=0.65,positions=np.arange(0,len(gene_list),1))

        for i,pc in enumerate(parts['bodies']):
            if gene_list[i]=="Control":
                pc.set_facecolor(gene_color_new["Control"])
            else:
                pc.set_facecolor("#999999")
            pc.set_edgecolor(black)
            pc.set_alpha(1)
            pc.set_linewidth(1.5)

        ax[0].boxplot(gene_groups,positions=np.arange(0,len(gene_list),1),showcaps=False,widths=0.06, patch_artist=True, boxprops=dict(color=black, facecolor=black),whiskerprops=dict(color=black, linewidth=2), medianprops=dict(color="w", linewidth=0 ),showfliers=False)
        inds = np.arange(0, len(quartiles))
        ax[0].scatter(inds, quartiles[:,1], marker='o', color='white', s=30, zorder=3)
        
        xticks_new=[]
        for i in gene_list:
            print(i,counts[i])
            if i != "Control":
                pval_prs=or_table_pr.loc[(or_table_pr[0]==i),4].reset_index()
                pval_prs=pval_prs.iloc[0,1]
            else:
                pval_prs=1
            if pval_prs <0.05:
                xticks_new.append(i[0].upper()+i[1:]+"\n("+str(counts[i])+")\n*")
            else:
                xticks_new.append(i[0].upper()+i[1:]+"\n("+str(counts[i])+")")
        print(xticks_new)
        ax[0].set_xticks(np.arange(0,len(gene_list),1))
        #ax[0].set_xticklabels(xticks_new,rotation=45)
        ax[0].set_xticklabels([])
        ax[0].tick_params(axis='y',labelsize=15,size=7)
        ax[0].tick_params(axis='x',labelsize=12,labelbottom=False)
        ax[0].grid(axis='x')
        ax[0].xaxis.label.set_size(20)
        ax[0].yaxis.label.set_size(22)
        ax[0].set_ylabel("PRS")
        ax[0].set_ylim([-4,4])
        #ax[0].set_xlabel("Gene")
        ax[0].axhline(y=temp.loc[temp["single_gene"]=="Control",y].median(), color='k', linestyle='--')
        print(temp.loc[temp["single_gene"]=="Control",y].median())
                        

#plt.savefig("../figures_new/prs_ch_mca_new.png",dpi=600,bbox_inches="tight")
#plt.savefig("../figures_new/prs_ch_mca_new.pdf",bbox_inches="tight")


or_table_pr=or_table_pr.set_index(0).loc[[x for x in gene_list if x!="Control"],:]
or_table_pr=or_table_pr.reset_index()
or_significant=or_table_pr.loc[(or_table_pr[4]>=0.05),:]
ax[1].errorbar(or_significant.index+1,or_significant.loc[:,1],yerr=[or_significant.loc[:,1]-or_significant.loc[:,2],or_significant.loc[:,3]-or_significant.loc[:,1]],fmt="o",markersize=15,elinewidth=1,c="#b3b3b3",label="P"+r'$\mathrm{\geq}0.05$')
or_significant=or_table_pr.loc[(or_table_pr[4]<0.05),:]
ax[1].errorbar(or_significant.index+1,or_significant.loc[:,1],yerr=[or_significant.loc[:,1]-or_significant.loc[:,2],or_significant.loc[:,3]-or_significant.loc[:,1]],fmt="o",markersize=15,elinewidth=1,c="#4d4d4d",label="P"+r'$\mathrm{<}0.05$')

ax[1].set_xticks(np.arange(0,len(gene_list)))
ax[1].set_xticklabels(xticks_new,fontsize=15,rotation=35)
ax[1].set_ylabel("OR",fontsize=22)
ax[1].set_xlabel("CH driver gene",fontsize=22)
ax[1].tick_params(axis='y',labelsize=15,size=7)
ax[1].axhline(y=1,linestyle="--",lw=1)
ax[1].legend()



for a in ax:
    a.spines[['right', 'top']].set_visible(False)

#plt.savefig("../figures_revision1/prs_ch_or_allgenes_singlemut.png",dpi=600,bbox_inches="tight")
#plt.savefig("../figures_revision1/prs_ch_or_allgenes_singlemut.pdf",bbox_inches="tight")
#plt.savefig("../figures_revision1/prs_ch_or_allgenes_singlemut.svg",bbox_inches="tight")

plt.show()
            


In [ ]:
data=ukb500k_ts_som[["Z-adjusted T/S log", "sex", "age", "smoking_status", "single_gene", "single_gene_vaf","PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10", "WBC_wins", "MO_perc_wins", "LY_perc_wins", "BA_perc_wins","NE_perc_wins","EO_perc_wins","eid","HM"]].copy(deep=True)
print(data.shape,data[data["HM"].isna()].shape)


data=data.dropna()
data=data[(data["HM"]==0)]
print(data.shape,data[data["HM"].isna()].shape)
print(data.shape)


data=data.rename(columns={"Z-adjusted T/S log":"tl"})
formula = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"])

lin_model=smf.ols(formula=formula,data=data)
result_fullreg=lin_model.fit(max_iterations= 100000)
display(result_fullreg.summary2())
params_fullreg = result_fullreg.params
conf_fullreg = result_fullreg.conf_int()

print(result_fullreg.pvalues)
data=data.merge(result_fullreg.resid.rename("residuals"),left_index=True,right_index=True)


In [ ]:
#pvalue_table=pd.read_csv("/home/sruthi/Downloads/2024-12-26_three-bin-comparison_pairwise-wilcox_bonferroni3.csv",usecols=["single_gene","vaf_group_comparison","p.value"],dtype="str")
#pvalue_table=pd.read_csv("~/Downloads/2025-01-24_single-gene_noHM_three-bin-comparison_pairwise-wilcox_bonferroni3.csv",usecols=["single_gene","vaf_group_comparison","adj.p.value"],dtype="str")

In [ ]:
y="residuals"
#pvalue_table=pd.read_csv("/home/sruthi/Downloads/2024-12-26_three-bin-comparison_pairwise-wilcox_bonferroni3.csv",usecols=["single_gene","vaf_group_comparison","p.value"])
pvalue_table=pd.read_csv("~/Downloads/2025-01-24_single-gene_noHM_three-bin-comparison_pairwise-wilcox_bonferroni3.csv",usecols=["single_gene","vaf_group_comparison","adj.p.value"],dtype="str")
pvalue_table=pvalue_table.rename(columns={"adj.p.value":"p.value"})
print(data.shape)
overlapping_bins=[0,0.1,0.2]
box_groups=[data.loc[(data["single_gene"]=="Control") & (data[y].notna()),y]]
tick_labels=["Control"]
box_colors=[gene_color["Control"]]
ticks=[-1+0.5]
j=0
plt.figure(figsize=(16,7))
ax=plt.gca()
box_color_pal=sns.color_palette("viridis", 3)
vaf_bin_counts=["Control\n("+str(box_groups[0].shape[0])+")"]
vaf_mean_all=[""]
for gene in ["ASXL1","CALR","CBL","DNMT3A_R882","DNMT3A_other","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1"]:
    temp=data[data["single_gene"]==gene]
    temp=temp[temp[y].notna()]
    print(temp["single_gene_vaf"])
 #   box_groups=[]
  #  tick_labels=[]
  #  ticks=[]
    gene_vaf_bin_counts=[]
    vaf_mean_all=[]
    
    df1=temp.loc[(temp["single_gene_vaf"]<=0.1),:]
    df2=temp.loc[((temp["single_gene_vaf"]>0.1)&(temp["single_gene_vaf"]<=0.2)),:]
    df3=temp.loc[(temp["single_gene_vaf"]>0.2),:]
    dfs=[df1,df2,df3]
    #vaf_mean_all=[df1["single_gene_vaf"].mean(),df2["single_gene_vaf"].mean(),df3["single_gene_vaf"].mean()]
    
    gene_pos=[]
    for n,df in enumerate(dfs):
        if df.shape[0]>10:
            box_groups.append(df[y])
            gene_vaf_bin_counts.append(str(df.shape[0]))
            vaf_mean_all.append("{:.2f}".format(df["single_gene_vaf"].mean()))
            box_colors.append(box_color_pal[n])    
            ticks.append(j+0.2+0.2*n)
            gene_pos.append(j+0.2+0.2*n)
    
    ypos={'1v2':3,'1v3':3.3,'2v3':3.6}
    for index, row in pvalue_table.loc[pvalue_table["single_gene"]==gene].iterrows():
        if float(row["p.value"])<0.05:        
            x=str(row["vaf_group_comparison"]).split("v")
            print (x)
            #plt.plot([gene_pos[int(x[0])-1],gene_pos[int(x[1])-1],[1,2]
            plt.plot([gene_pos[int(x[0])-1],gene_pos[int(x[1])-1]],[ypos[row["vaf_group_comparison"]],ypos[row["vaf_group_comparison"]]],c="k",linewidth=0.5)
            print (gene_pos[int(x[1])-1]-gene_pos[int(x[0])-1])
            if float(row["p.value"])<0.001:
                ax.text((gene_pos[int(x[1])-1]+gene_pos[int(x[0])-1])/2,ypos[row["vaf_group_comparison"]]+0.05,"***",horizontalalignment='center',verticalalignment='center')
            #ax.text((gene_pos[int(x[1])-1]+gene_pos[int(x[0])-1])/2,3.1,"*")
            elif float(row["p.value"])<0.01:
                ax.text((gene_pos[int(x[1])-1]+gene_pos[int(x[0])-1])/2,ypos[row["vaf_group_comparison"]]+0.05,"**",horizontalalignment='center',verticalalignment='center')
            #ax.text((gene_pos[int(x[1])-1]+gene_pos[int(x[0])-1])/2,3.1,"*")
            else:
                ax.text((gene_pos[int(x[1])-1]+gene_pos[int(x[0])-1])/2,ypos[row["vaf_group_comparison"]]+0.05,"*",horizontalalignment='center',verticalalignment='center')
            #ax.text((gene_pos[int(x[1])-1]+gene_pos[int(x[0])-1])/2,3.1,"*")
            

            
            
        """
        if df.shape[0]>10:
            box_groups.append(df)
            if i != 0.3:
                tick_labels.append("["+ "{:1.1f}".format(i)+ ","+"{:1.1f}".format(i+0.2)+ ")\n("+ str(box_groups[-1].shape[0])+")")
            else:
                tick_labels.append("[0.3-1]"+"\n("+ str(box_groups[-1].shape[0])+")")
            box_colors.append(box_color_pal[int(i*10)])    
            ticks.append(j+i*10*0.15)
        gene_vaf_bin_counts.append(str(df.shape[0]))
        vaf_mean_all.append("{:.2f}".format(vaf_mean))
        """
    vaf_bin_counts.append(gene+"\n("+",".join(gene_vaf_bin_counts)+")")
    #b=plt.boxplot(box_groups,positions=np.arange(0,len(box_groups),1))
    j+=1
bp=plt.boxplot(box_groups,positions=ticks,widths = 0.08,flierprops = {'marker':'o','markersize':1,'markerfacecolor':'none','markeredgewidth':0.1}, boxprops = dict( linestyle='none'),whiskerprops={'lw':0.3}, medianprops={'color':'r'},capprops={'linewidth':0.5},patch_artist=True)

        

ax=plt.gca()
xticks=ax.get_xticklabels()
plt.xticks(np.arange(-1,14,1)+0.5,["Control","ASXL1","CALR","CBL","DNMT3A_R882","DNMT3A_other","GNAS","GNB1","JAK2","PPM1D","SF3B1","SRSF2","TET2","TP53","U2AF1"],fontsize=6)
plt.xticks(np.arange(-1,14,1)+0.5,vaf_bin_counts,fontsize=6)
i=0
for patch in bp['boxes']:
        patch.set(facecolor=box_colors[i]) 
        patch.set(edgecolor=box_colors[i])
        i+=1

plt.ylim([-5,5])
#    plt.title(gene+"("+str(temp.shape[0])+")")
#    plt.show()

legend_patch = [mpatches.Patch(color=box_color_pal[0], label='<=0.1'),mpatches.Patch(color=box_color_pal[1], label='0.1-0.2'),mpatches.Patch(color=box_color_pal[2], label='>0.2')]
plt.legend(handles=legend_patch,title="VAF bins",fontsize=6,title_fontsize=6,ncol=2,loc='upper left',bbox_to_anchor=(0.04, 0.995))
plt.yticks(fontsize=8)
plt.axhline(y=data.loc[data["single_gene"]=="Control",y].median(), color='k', linestyle='--',linewidth=0.5)
plt.ylabel("Telomere length",fontsize=10)
ax.text(0.2,0.95,"*   P<0.05\n**   P<0.01\n***  P<0.001",fontsize=6,transform=ax.transAxes)

#plt.savefig("../figures_revision1/vaf_vs_tl_multibin_reviewer1_noHM.png",bbox_inches="tight",dpi=600)
#plt.savefig("../figures_revision1/vaf_vs_tl_multibin_reviewer1_noHM.pdf",bbox_inches="tight")
#plt.savefig("../figures_revision1/vaf_vs_tl_multibin_reviewer1_noHM.svg",bbox_inches="tight")

plt.show()

In [ ]:
def bin_vaf(v):
    if v != "NA":
        if v<=0.1:
            return 1
        elif v>0.1 and v<=0.2:
            return 2
        else:
            return 3
        
data["vaf_group"]=data["single_gene_vaf"].map(lambda row:bin_vaf(row))

In [ ]:
#data[["eid","residuals","single_gene","single_gene_vaf","vaf_group"]].to_csv("ltl_residual_multibin_vaf_singlegene.tsv",sep="\t",index=False,na_rep="NA")


#data[["eid","residuals","single_gene","single_gene_vaf","vaf_group"]].to_csv("ltl_residual_multibin_vaf_singlegene_noHM.tsv",sep="\t",index=False,na_rep="NA")



In [ ]:
######## LTL~PRS+VAF+PRS*VAF ########## 
####### gene specific models for LTL ##############

temp=ukb500k_ts_som.copy(deep=True)
y="tl"
temp= temp.rename(columns={'Z-adjusted T/S log': 'tl'})
temp=temp.loc[(temp[y].notna())]

data=temp[["total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","single_gene","single_gene_vaf","WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins","tl","HM"]].copy(deep=True)
data=data.dropna()
data=data[(data["HM"]==0)]
total_prs=np.arange(-10,10.25,0.25)
vaf_values=np.arange(0.05,1.05,0.05)
inputs_base=pd.DataFrame(list(product(total_prs,vaf_values)))
inputs_base.columns=["total_PRS","single_gene_vaf"]

for col in data.columns:
    if col not in (["total_PRS","single_gene_vaf","tl","sex","smoking_status","single_gene","HM"]):
        inputs_base[col]=data[col].mean()
print(inputs_base.columns)
all_inputs=[]

for values in data["sex"].unique():
    inputs_base["sex"]=values
    for ss in data["smoking_status"].unique():
        inputs_base["smoking_status"]=ss
        all_inputs.append(inputs_base.copy(deep=True))
all_inputs=pd.concat(all_inputs).reset_index(drop=True)


formula_0 = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"])

formula_1 = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','total_PRS',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"])

formula_2 = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"single_gene_vaf","WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"])

formula_3 = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','total_PRS',"single_gene_vaf","WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"])

formula_4 = 'tl ~' + ' + '.join(['C(sex,Treatment(reference="Female"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','total_PRS',"single_gene_vaf","WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins"]) + "+ total_PRS * single_gene_vaf"

gene_list=list(temp["single_gene"].unique())
gene_list.remove("Control")
gene_order={"DNMT3A_other":0,"TET2":1,"SRSF2":2,"PPM1D":3}

coef_list_all=[]
adj_r_all=[]
g=0
max_tl,min_tl=[],[]
tl_sens_all={}
for gene in gene_list:
    data=temp[["total_PRS", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","single_gene","single_gene_vaf","WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins","tl","HM"]].copy(deep=True)
    data=data.dropna()
    data=data[(data["HM"]==0)]
    
    print(data.shape)
    data=data[(data["single_gene"]==gene)]
    data["single_gene_vaf"]=data["single_gene_vaf"].astype(float)
    data["total_PRS"]=data["total_PRS"].astype(float)
    print(data["single_gene"].value_counts())
    
    adj_r=[gene]
    coef_list=[gene]
    
    lin_model=smf.ols(formula=formula_0,data=data)
    result=lin_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = result.params
    conf = result.conf_int()
    print(result.pvalues)
    adj_r+=[result.rsquared_adj]
    
    
    lin_model=smf.ols(formula=formula_1,data=data)
    result=lin_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = result.params
    conf = result.conf_int()
    adj_r+=[result.rsquared_adj]
    coef_list+=[params["total_PRS"],conf.loc["total_PRS",0],conf.loc["total_PRS",1],result.pvalues["total_PRS"]]
    
    lin_model=smf.ols(formula=formula_2,data=data)
    result=lin_model.fit(max_iterations= 100000)
    display(result.summary2())
    params = result.params
    conf = result.conf_int()
    adj_r+=[result.rsquared_adj]
    coef_list+=[params["single_gene_vaf"],conf.loc["single_gene_vaf",0],conf.loc["single_gene_vaf",1],result.pvalues["single_gene_vaf"]]
    
    lin_model_3=smf.ols(formula=formula_3,data=data)
    result_3=lin_model_3.fit(max_iterations= 100000)
    display(result_3.summary2())
    params = result_3.params
    print(params)
    conf = result_3.conf_int()
    adj_r+=[result_3.rsquared_adj]
    coef_list+=[params["total_PRS"],conf.loc["total_PRS",0],conf.loc["total_PRS",1],result_3.pvalues["total_PRS"],params["single_gene_vaf"],conf.loc["single_gene_vaf",0],conf.loc["single_gene_vaf",1],result_3.pvalues["single_gene_vaf"]]
    
    lin_model_4=smf.ols(formula=formula_4,data=data)
    result_4=lin_model_4.fit(max_iterations= 100000)
    display(result_4.summary2())
    params = result_4.params
    conf = result_4.conf_int()
    coef_list+=[params["total_PRS"],conf.loc["total_PRS",0],conf.loc["total_PRS",1],result_4.pvalues["total_PRS"],params["single_gene_vaf"],conf.loc["single_gene_vaf",0],conf.loc["single_gene_vaf",1],result_4.pvalues["single_gene_vaf"],params["total_PRS:single_gene_vaf"],conf.loc["total_PRS:single_gene_vaf",0],conf.loc["total_PRS:single_gene_vaf",1],result_4.pvalues["total_PRS:single_gene_vaf"]]
    anovaResults = anova_lm(result_3, result_4)
    adj_r+=[result_4.rsquared_adj,anovaResults.loc[1,"Pr(>F)"]]
    print(anovaResults)
    adj_r_all.append(adj_r)
    coef_list_all.append(coef_list)
    
    if gene in ["DNMT3A_other","TET2","SRSF2","PPM1D"]:
        tl_pred=(result_3.predict(all_inputs)).reset_index(drop=True)
        max_tl.append(tl_pred.max())
        min_tl.append(tl_pred.min())
        tl_sens=pd.concat([all_inputs,tl_pred],axis=1).rename(columns={0:"pred"})
        tl_sens=tl_sens.groupby(["total_PRS","single_gene_vaf"]).agg({"pred":'mean'}).reset_index()
        tl_sens=tl_sens.pivot_table(columns=["total_PRS"],index=["single_gene_vaf"])
        tl_sens_all[gene]=tl_sens

adj_r_all=pd.DataFrame(adj_r_all)
adj_r_all.columns=["Gene","Basic","PRS","VAF","PRS+VAF","PRS_VAF_interaction","ANOVA"]

coef_list_all=pd.DataFrame(coef_list_all)
coef_list_all.columns=["Gene","PRS_only_coef","PRS_only_coef_lowerci","PRS_only_coef_higherci","PRS_only_pvalue","VAF_only_coef","VAF_only_coef_lowerci","VAF_only_coef_higherci","VAF_only_pvalue","PRS_plus_VAF_PRS_coef","PRS_plus_VAF_PRS_coef_lowerci","PRS_plus_VAF_PRS_coef_higherci","PRS_plus_VAF_PRS_pvalue","PRS_plus_VAF_VAF_coef","PRS_plus_VAF_VAF_coef_lowerci","PRS_plus_VAF_VAF_coef_higherci","PRS_plus_VAF_VAF_pvalue","PRS_VAF_int_PRS_coef","PRS_VAF_int_PRS_coef_lowerci","PRS_VAF_int_PRS_coef_higherci","PRS_VAF_int_PRS_pvalue","PRS_VAF_int_VAF_coef","PRS_VAF_int_VAF_coef_lowerci","PRS_VAF_int_VAF_coef_higherci","PRS_VAF_int_VAF_pvalue","PRS_VAF_int_PRS:VAF_coef","PRS_VAF_int_PRS:VAF_coef_lowerci","PRS_VAF_int_PRS:VAF_coef_higherci","PRS_VAF_int_PRS:VAF_pvalue"]

display(coef_list_all)

display(adj_r_all)


In [ ]:
###### Sensitivity analysis figure ##############

fig, axes = plt.subplots(2, 2,sharey=True,sharex=True,figsize=(12,12))#,gridspec_kw={'height_ratios':[5,5,1.5],'width_ratios':[15,4]})0
axes=[axes[0,0],axes[0,1],axes[1,0],axes[1,1]]
cbar_ax = fig.add_axes([0.93, 0.15, 0.02, 0.7])        

cmap = matplotlib.cm.YlGnBu_r
norm = matplotlib.colors.Normalize(vmin=min(min_tl), vmax=max(max_tl))

fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=cbar_ax,label='LTL')


for g,gene in enumerate(["DNMT3A_other","TET2","SRSF2","PPM1D"]):
    #if gene != "PPM1D":
    cbar=sns.heatmap(np.asarray(tl_sens_all[gene][::-1]),ax=axes[g], cbar=False,cmap=cmap,norm=norm, clip_on=False,linecolor=None,linewidths=0.0, rasterized=True)
    #else:
        #cbar=sns.heatmap(np.asarray(tl_sens_all[gene][::-1]),ax=axes[g],cmap=cmap,norm=norm,clip_on=False,cbar=False)
    axes[g].set_xticks(np.arange(0,84,8)+0.5,np.arange(-10,11,2))
    axes[g].set_yticks(np.arange(0,20,4)+0.5,[1,0.8,0.6,0.4,0.2])
    #fig.colorbar(cbar,ax=cbar_ax)
    axes[g].set_title(gene)

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.grid(False)
plt.xlabel("PRS",labelpad=20,fontsize=20)
plt.ylabel("VAF",labelpad=20,fontsize=20)

plt.savefig("../figures_revision1/sensitivity_analysis.pdf",bbox_inches="tight",dpi=600)
plt.savefig("../figures_revision1/sensitivity_analysis.svg",bbox_inches="tight")
plt.savefig("../figures_revision1/sensitivity_analysis.png",bbox_inches="tight")


In [ ]:
adj_r_all

In [ ]:
for pval in ["PRS_only_pvalue","PRS_plus_VAF_PRS_pvalue","PRS_plus_VAF_VAF_pvalue","PRS_VAF_int_PRS_pvalue","PRS_VAF_int_VAF_pvalue","PRS_VAF_int_PRS:VAF_pvalue"]:
    coef_list_all[pval+"_FDR"]=multipletests(coef_list_all[pval], alpha=0.05, method='fdr_bh',returnsorted=False,is_sorted=False)[1]

In [ ]:
#adj_r_all.to_csv("adj_r_singlegene_noHM.tsv",sep="\t",index=False)

In [ ]:
#coef_list_all[["Gene","PRS_only_coef","PRS_only_pvalue","VAF_only_coef","VAF_only_pvalue","PRS_plus_VAF_PRS_coef","PRS_plus_VAF_PRS_pvalue","PRS_plus_VAF_VAF_coef","PRS_plus_VAF_VAF_pvalue","PRS_VAF_int_PRS_coef","PRS_VAF_int_PRS_pvalue","PRS_VAF_int_VAF_coef","PRS_VAF_int_VAF_pvalue","PRS_VAF_int_PRS:VAF_coef","PRS_VAF_int_PRS:VAF_pvalue"]].to_csv("prs_vaf_interaction_singlegene_noHM.tsv",sep="\t",index=False)

In [ ]:
coef_list_all[["Gene","PRS_only_coef","PRS_only_pvalue","PRS_only_pvalue_FDR","PRS_plus_VAF_PRS_coef","PRS_plus_VAF_PRS_pvalue","PRS_plus_VAF_PRS_pvalue_FDR","PRS_plus_VAF_VAF_coef","PRS_plus_VAF_VAF_pvalue","PRS_plus_VAF_VAF_pvalue_FDR","PRS_VAF_int_PRS_coef","PRS_VAF_int_PRS_pvalue","PRS_VAF_int_PRS_pvalue_FDR","PRS_VAF_int_VAF_coef","PRS_VAF_int_VAF_pvalue","PRS_VAF_int_VAF_pvalue_FDR","PRS_VAF_int_PRS:VAF_coef","PRS_VAF_int_PRS:VAF_pvalue", "PRS_VAF_int_PRS:VAF_pvalue_FDR"]].to_csv("prs_vaf_interaction_singlegene_noHM_FDR.tsv",sep="\t",index=False)

In [ ]:
########### Double mutant analysis ###########
########### Regression for LTL~covariates so as to get the adjusted LTL ##########
y='Z-adjusted T/S log'
temp=ukb500k_ts_som_multigene[["Z-adjusted T/S log", "sex", "age", "smoking_status", "PC1", "PC2", "PC3", "PC4","PC5","PC6","PC7","PC8","PC9","PC10","multi_gene_TET2","total_PRS","WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins","eid","HM"]].copy(deep=True)
print(temp.shape)

formula = 'tl~' + ' + '.join(['C(sex,Treatment(reference="Male"))','age','C(smoking_status,Treatment(reference="Never"))','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10',"WBC_wins","MO_perc_wins","LY_perc_wins","BA_perc_wins","NE_perc_wins","EO_perc_wins","total_PRS"])


temp=temp.rename(columns={"Z-adjusted T/S log":"tl"})
temp=temp.dropna()
temp=temp[(temp["HM"]==0)]
lin_model=smf.ols(formula=formula,data=temp)
result=lin_model.fit(max_iterations= 100000)
display(result.summary2())
params = result.params
conf = result.conf_int()
print(result.pvalues)
temp=temp.merge(result.resid.rename("tl_residuals"),left_index=True,right_index=True)


In [ ]:
####### Get the differences in the VAF  between DNMT3A and TET2 clones ########

def get_vaf_diff(data_ukb):
    d=data_ukb.copy(deep=True)
    d=d[(d["multi_gene_TET2"]=="DNMT3A,TET2-TET2") | (d["multi_gene_TET2"]=="DNMT3A,TET2-DNMT3A")]
    d["tet2_dnmt3a_vaf"]=0
    d["dnmt3a_tet2_vaf"]=0
    d["dnmt3a_tet2_diff"]=0
    for index, row in d.iterrows():
        v=np.array(row["vaf"].split(",")).astype(float)
        all_genes_present=np.array(row["symbol"].split(","))
        d.loc[index,"tet2_dnmt3a_vaf"]=np.max(v[all_genes_present=="DNMT3A"])
        d.loc[index,"dnmt3a_tet2_vaf"]=np.max(v[all_genes_present=="TET2"])
        d.loc[index,"dnmt3a_tet2_diff"]=np.max(v[all_genes_present=="DNMT3A"])-np.max(v[all_genes_present=="TET2"])
    data_ukb=data_ukb.merge(d[["eid","dnmt3a_tet2_vaf","tet2_dnmt3a_vaf","dnmt3a_tet2_diff"]],on="eid",how='left')
    
    d=data_ukb.copy(deep=True)
    d=d[(d["multi_gene_TET2"]=="SRSF2,TET2-TET2") | (d["multi_gene_TET2"]=="SRSF2,TET2-SRSF2")]
    d["tet2_srsf2_vaf"]=0
    d["srsf2_tet2_vaf"]=0
    d["srsf2_tet2_diff"]=0
    for index, row in d.iterrows():
        v=np.array(row["vaf"].split(",")).astype(float)
        all_genes_present=np.array(row["symbol"].split(","))
        d.loc[index,"tet2_srsf2_vaf"]=np.max(v[all_genes_present=="SRSF2"])
        d.loc[index,"srsf2_tet2_vaf"]=np.max(v[all_genes_present=="TET2"])
        d.loc[index,"srsf2_tet2_diff"]=np.max(v[all_genes_present=="SRSF2"])-np.max(v[all_genes_present=="TET2"])
    data_ukb=data_ukb.merge(d[["eid","srsf2_tet2_vaf","tet2_srsf2_vaf","srsf2_tet2_diff"]],on="eid",how='left')
    return data_ukb

In [ ]:
vaf_diff=get_vaf_diff(ukb500k_ts_som_multigene)

In [ ]:
########### Plot the adjusted residuals ############
fig, axes = plt.subplots(1, 2,sharex=False,sharey=True,figsize=(10,4))
eid_sel=vaf_diff.loc[((vaf_diff["srsf2_tet2_diff"]<=-0.035) | (vaf_diff["srsf2_tet2_diff"]>=0.035)) & (vaf_diff["srsf2_tet2_vaf"]<0.6) & (vaf_diff["tet2_srsf2_vaf"]<0.6),"eid"]

bp = temp[((temp["multi_gene_TET2"]=="SRSF2,TET2-SRSF2") | (temp["multi_gene_TET2"]=="SRSF2,TET2-TET2")) & (temp["eid"].isin(eid_sel))].boxplot(column='tl_residuals', by='multi_gene_TET2', grid=False,ax=axes[0],widths=0.5,boxprops={'color':'#b3cce6','facecolor':'#b3cce6'}, patch_artist=True,zorder=1,medianprops={'color':'r'},whiskerprops={'color':'k'})
g=["SRSF2,TET2-SRSF2","SRSF2,TET2-TET2"]

#vaf_diff.loc[((vaf_diff["srsf2_tet2_diff"]<=-0.035) | (vaf_diff["srsf2_tet2_diff"]>=0.035)) & (vaf_diff["srsf2_tet2_vaf"]<0.6) & (vaf_diff["tet2_srsf2_vaf"]<0.6) & (vaf_diff["eid"].isin(temp["eid"])),["eid","symbol","vaf","srsf2_tet2_vaf","tet2_srsf2_vaf","srsf2_tet2_diff"]].to_csv("srsf2_tet2_data.tsv",sep="\t",index=False,header=["eid","symbol","vaf","tet2_vaf","srsf2_vaf","srsf2_tet2_diff"])

for i in range(1,3):
    y = temp.loc[(temp["multi_gene_TET2"]==g[i-1]) & (temp["eid"].isin(eid_sel)),"tl_residuals"].dropna()
    print(y.shape)
    np.random.seed(0)
    # Add some random "jitter" to the x-axis
    z = np.random.normal(i, 0.06, size=len(y))
    #axes[0].scatter(z, y, alpha=0.5,s=20,edgecolor='#800000',facecolors='None',linewidths=1.5)
axes[0].set_xticks([1,2])
axes[0].set_xticklabels(["SRSF2","TET2"],fontsize=11)
axes[0].set_ylim([-4,4])
axes[0].set_ylabel("LTL",fontsize=13)
axes[0].set_xlabel("Gene with larger clone",fontsize=13)
axes[0].set_title("")
axes[0].text(2,3,"P=0.01",fontsize=12)
axes[0].text(0.3,4.5,"a",fontsize=15,fontweight="bold")



eid_sel=vaf_diff.loc[((vaf_diff["dnmt3a_tet2_diff"]<=-0.035) | (vaf_diff["dnmt3a_tet2_diff"]>=0.035)) & (vaf_diff["dnmt3a_tet2_vaf"]<0.6) & (vaf_diff["tet2_dnmt3a_vaf"]<0.6),"eid"]
bp = temp[((temp["multi_gene_TET2"]=="DNMT3A,TET2-DNMT3A") | (temp["multi_gene_TET2"]=="DNMT3A,TET2-TET2")) & (temp["eid"].isin(eid_sel))].boxplot(column='tl_residuals', by='multi_gene_TET2', grid=False,ax=axes[1],widths=0.5,boxprops={'color':'#b3cce6','facecolor':'#b3cce6'}, patch_artist=True,zorder=1,medianprops={'color':'r'},whiskerprops={'color':'k'},)

print(temp.loc[((temp["multi_gene_TET2"]=="DNMT3A,TET2-DNMT3A") | (temp["multi_gene_TET2"]=="DNMT3A,TET2-TET2")) & (temp["eid"].isin(eid_sel)),"multi_gene_TET2"].value_counts())

g=["DNMT3A,TET2-DNMT3A","DNMT3A,TET2-TET2"]

#vaf_diff.loc[((vaf_diff["dnmt3a_tet2_diff"]<=-0.035) | (vaf_diff["dnmt3a_tet2_diff"]>=0.035)) & (vaf_diff["dnmt3a_tet2_vaf"]<0.6) & (vaf_diff["tet2_dnmt3a_vaf"]<0.6) & (vaf_diff["eid"].isin(temp["eid"])),["eid","symbol","vaf","dnmt3a_tet2_vaf","tet2_dnmt3a_vaf","dnmt3a_tet2_diff"]].to_csv("dnmt3a_tet2_data.tsv",sep="\t",index=False,header=["eid","symbol","vaf","tet2_vaf","dnmt3a_vaf","dnmt3a_tet2_diff"])
             
for i in range(1,3):
    y = temp.loc[(temp["multi_gene_TET2"]==g[i-1]) & (temp["eid"].isin(eid_sel)),"tl_residuals"].dropna()
    np.random.seed(0)
    # Add some random "jitter" to the x-axis
    z = np.random.normal(i, 0.06, size=len(y))
    #axes[1].scatter(z, y, alpha=0.5,s=20,edgecolor='#800000',facecolors='None',linewidths=1.5)
axes[1].set_xticks([1,2])
axes[1].set_xticklabels(["DNMT3A","TET2"],fontsize=11)
axes[1].set_xlabel("Gene with larger clone",fontsize=13)
axes[1].set_title("")
axes[1].text(2,3,"P=0.90",fontsize=12)
axes[1].text(0.4,4.5,"b",fontsize=15,fontweight="bold")
#axes[1].set_yticks(np.arange(-4,5,1))
axes[1].set_facecolor('white')
fig.suptitle('')
plt.subplots_adjust(wspace=0.1)

#plt.savefig("../figures_revision1/double_mut_comparison_noHM.pdf",bbox_inches="tight")
#plt.savefig("../figures_revision1/double_mut_comparison.png",bbox_inches="tight",dpi=600)
#plt.savefig("../figures_revision1/double_mut_comparison.svg",bbox_inches="tight")

In [ ]:
################ Compare the adjusted LTLs - srsf2, tet2 ##############
eid_sel=vaf_diff.loc[((vaf_diff["srsf2_tet2_diff"]<=-0.035) | (vaf_diff["srsf2_tet2_diff"]>=0.035)) & (vaf_diff["srsf2_tet2_vaf"]<0.6) & (vaf_diff["tet2_srsf2_vaf"]<0.6),"eid"]
print(eid_sel.shape)
print(temp.loc[((temp["multi_gene_TET2"]=="SRSF2,TET2-SRSF2") & (temp["eid"].isin(eid_sel))),"tl_residuals"].shape,temp.loc[((temp["multi_gene_TET2"]=="SRSF2,TET2-TET2") & (temp["eid"].isin(eid_sel))),"tl_residuals"].shape)

mannwhitneyu(temp.loc[((temp["multi_gene_TET2"]=="SRSF2,TET2-SRSF2") & (temp["eid"].isin(eid_sel))),"tl_residuals"],temp.loc[((temp["multi_gene_TET2"]=="SRSF2,TET2-TET2") & (temp["eid"].isin(eid_sel))),"tl_residuals"])


In [ ]:
################ Compare the adjusted LTLs - dnmt3a, tet2 ##############

eid_sel=vaf_diff.loc[((vaf_diff["dnmt3a_tet2_diff"]<=-0.035) | (vaf_diff["dnmt3a_tet2_diff"]>=0.035)) & (vaf_diff["dnmt3a_tet2_vaf"]<0.6) & (vaf_diff["tet2_dnmt3a_vaf"]<0.6),"eid"]

print(temp.loc[((temp["multi_gene_TET2"]=="DNMT3A,TET2-DNMT3A") & (temp["eid"].isin(eid_sel))),"tl_residuals"].shape, temp.loc[((temp["multi_gene_TET2"]=="DNMT3A,TET2-TET2") & (temp["eid"].isin(eid_sel))),"tl_residuals"].shape)

mannwhitneyu(temp.loc[((temp["multi_gene_TET2"]=="DNMT3A,TET2-DNMT3A") & (temp["eid"].isin(eid_sel))),"tl_residuals"], temp.loc[((temp["multi_gene_TET2"]=="DNMT3A,TET2-TET2") & (temp["eid"].isin(eid_sel))),"tl_residuals"])
